# Анализ данных вакансий Аналитиков данных и Бизнес-аналитиков с сайта HH.ru

Цель данного исследования - выявить различия в вакансиях Аналитиков данных и Бизнес-аналитиков.
Будут рассмотрены различия вакансий в трёх аспектах:
1. Требования к кандидатам: Наиболее желательные hard skills и soft skills. Опыт работы.
2. Типиичные места и условия работы: Топ-работодатели, уровень заработной платы, типичные виды занятости и расписания.
3. Динамика количества вакансий по месяцам.

Каждый пункт будет рассмотрен в разрезе четырёх грейдов: Junior, Junior+, Middle и Senior.


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

import pandas as pd
import seaborn as sns
from datetime import datetime, timedelta
from matplotlib import pyplot as plt
import numpy as np
from collections import Counter
import re

da_raw = pd.read_csv('gdrive/My Drive/vacancies_da_new.csv')
ba_raw = pd.read_csv('gdrive/My Drive/vacancies_ba_new.csv')

Mounted at /content/gdrive


Данные для анализа вакансий взяты с сайта hh.ru.

## Предобработка DA

In [ ]:
da_raw.head()

,id,name,published_at,alternate_url,type,employer,department,area,experience,key_skills,schedule,employment,description,description_lemmatized,salary_from,salary_to,salary_bin,key_skills_from_key_skills_field,hard_skills_from_description,soft_skills_from_description
0,50293361,Аналитик данных (Data analyst),2024-01-29 12:16:08,https://hh.ru/vacancy/50293361,Открытая,"Красное & Белое, розничная сеть",NaN,Москва,Junior+ (1-3 years),"['MS SQL', 'Power Pivot', 'Прогнозирование', '...",Полный день,Полная занятость,для решение разнообразных и интересных задач в...,решение разнообразный интересный задача команд...,NaN,NaN,ЗП не указана,"SQL, VBA, MS SQL, MS PowerPoint, Анализ данных...",NaN,Коммуникация
1,79046568,Data Analyst / Специалист по веб-аналитике,2023-11-03 09:49:29,https://hh.ru/vacancy/79046568,Открытая,НОНТОН.РФ,NaN,Санкт-Петербург,Junior+ (1-3 years),"['Яндекс.Директ', 'Маркетинговый анализ', 'Кон...",Полный день,Полная занятость,крупная мебельная компания в поиске web-аналит...,крупный мебельный компания поиск web аналитик ...,120000.0,200000.0,От 200 тысяч до 300 тысяч,"Google Analytics, Яндекс.Метрика, Веб-аналитик...",NaN,NaN
2,79115520,Senior Data Analyst,2023-10-15 10:00:10,https://hh.ru/vacancy/79115520,Открытая,СБЕР,Сбер. Data Science,Москва,Middle (3-6 years),[],Полный день,Полная занятость,команда сеть продаж сбера ищет руководителя на...,команда сеть продажа сбер искать руководитель ...,NaN,NaN,ЗП не указана,NaN,NaN,NaN
3,79598893,Data Analyst (SQL),2023-12-04 09:31:34,https://hh.ru/vacancy/79598893,Открытая,WILDBERRIES,NaN,Москва,Junior+ (1-3 years),"['SQL', 'Управленческая отчетность', 'OLAP', '...",Полный день,Полная занятость,wildberries — это крупнейшая в странах еаэс ме...,wildberries крупный страна еаэс международный ...,NaN,NaN,ЗП не указана,"SQL, Управленческая отчетность, SQL запросы, E...",NaN,NaN
4,80616218,Data Analyst,2023-12-03 14:43:59,https://hh.ru/vacancy/80616218,Открытая,Контакт,NaN,Санкт-Петербург,Junior+ (1-3 years),"['SQL', 'confluence', 'Документация', 'Jira', ...",Полный день,Полная занятость,мы строим современную и технологичную финтех-к...,строить современный технологичный финтех компа...,NaN,NaN,ЗП не указана,"Power BI, SQL, Jira, Confluence",confluence,Документация


In [ ]:
#уберем некоторые столбцы, которые нам не пригодятся при анализе

da = da_raw.drop(['alternate_url', 'type', 'department'], axis=1)
da.head()

,id,name,published_at,employer,area,experience,key_skills,schedule,employment,description,description_lemmatized,salary_from,salary_to,salary_bin,key_skills_from_key_skills_field,hard_skills_from_description,soft_skills_from_description
0,50293361,Аналитик данных (Data analyst),2024-01-29 12:16:08,"Красное & Белое, розничная сеть",Москва,Junior+ (1-3 years),"['MS SQL', 'Power Pivot', 'Прогнозирование', '...",Полный день,Полная занятость,для решение разнообразных и интересных задач в...,решение разнообразный интересный задача команд...,NaN,NaN,ЗП не указана,"SQL, VBA, MS SQL, MS PowerPoint, Анализ данных...",NaN,Коммуникация
1,79046568,Data Analyst / Специалист по веб-аналитике,2023-11-03 09:49:29,НОНТОН.РФ,Санкт-Петербург,Junior+ (1-3 years),"['Яндекс.Директ', 'Маркетинговый анализ', 'Кон...",Полный день,Полная занятость,крупная мебельная компания в поиске web-аналит...,крупный мебельный компания поиск web аналитик ...,120000.0,200000.0,От 200 тысяч до 300 тысяч,"Google Analytics, Яндекс.Метрика, Веб-аналитик...",NaN,NaN
2,79115520,Senior Data Analyst,2023-10-15 10:00:10,СБЕР,Москва,Middle (3-6 years),[],Полный день,Полная занятость,команда сеть продаж сбера ищет руководителя на...,команда сеть продажа сбер искать руководитель ...,NaN,NaN,ЗП не указана,NaN,NaN,NaN
3,79598893,Data Analyst (SQL),2023-12-04 09:31:34,WILDBERRIES,Москва,Junior+ (1-3 years),"['SQL', 'Управленческая отчетность', 'OLAP', '...",Полный день,Полная занятость,wildberries — это крупнейшая в странах еаэс ме...,wildberries крупный страна еаэс международный ...,NaN,NaN,ЗП не указана,"SQL, Управленческая отчетность, SQL запросы, E...",NaN,NaN
4,80616218,Data Analyst,2023-12-03 14:43:59,Контакт,Санкт-Петербург,Junior+ (1-3 years),"['SQL', 'confluence', 'Документация', 'Jira', ...",Полный день,Полная занятость,мы строим современную и технологичную финтех-к...,строить современный технологичный финтех компа...,NaN,NaN,ЗП не указана,"Power BI, SQL, Jira, Confluence",confluence,Документация


In [ ]:
da.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 786 entries, 0 to 785
Data columns (total 17 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   id                                786 non-null    int64  
 1   name                              786 non-null    object 
 2   published_at                      786 non-null    object 
 3   employer                          786 non-null    object 
 4   area                              786 non-null    object 
 5   experience                        786 non-null    object 
 6   key_skills                        786 non-null    object 
 7   schedule                          786 non-null    object 
 8   employment                        786 non-null    object 
 9   description                       786 non-null    object 
 10  description_lemmatized            786 non-null    object 
 11  salary_from                       69 non-null     float64
 12  salary_t

Проведём предобработку данных во фрейме da

Изучим данные в числовых столбцах

In [ ]:
da.describe()

,id,salary_from,salary_to
count,7.860000e+02,69.000000,86.000000
mean,8.933467e+07,167361.594203,242533.720930
std,2.441172e+06,78265.805651,125657.467989
min,5.029336e+07,20000.000000,40000.000000
25%,8.811736e+07,100000.000000,123662.500000
50%,8.946152e+07,180000.000000,245000.000000
75%,9.080152e+07,233750.000000,327250.000000
max,9.241885e+07,398000.000000,561000.000000


Проверим дубликаты

In [ ]:
da.duplicated().sum()
#явных дубликатов нет

0

In [ ]:
print(da['area'].unique())
print(da['experience'].unique())
print(da['schedule'].unique())
print(da['employment'].unique())

['Москва' 'Санкт-Петербург' 'Иваново (Ивановская область)' 'Тбилиси'
 'Алматы' 'Новосибирск' 'Минск' 'Владивосток' 'Ташкент' 'Ростов-на-Дону'
 'Казань' 'Химки' 'Екатеринбург' 'Тюмень' 'Армения' 'Астана' 'Сербия'
 'Черногория' 'Кипр' 'Краснодар' 'Батуми' 'Баку' 'Атырау' 'Турция'
 'Аксай (Казахстан)' 'Бишкек' 'Рязань' 'Самара' 'Нижний Новгород'
 'Красноярск' 'Сочи' 'Томск' 'Ижевск' 'Иннополис']
['Junior+ (1-3 years)' 'Middle (3-6 years)' 'Senior (6+ years)'
 'Junior (no experince)']
['Полный день' 'Удаленная работа' 'Гибкий график' 'Сменный график']
['Полная занятость' 'Частичная занятость' 'Стажировка' 'Проектная работа']


Преобразуем форматы данных

In [ ]:
da['published_at'] = pd.to_datetime(da['published_at']).dt.date
#da['published_at'].unique()

Cоздадим колонку с месяцем опубликования вакансии

In [ ]:
da['month'] = pd.DatetimeIndex(da['published_at'])
da['month'] = da['month'].dt.strftime('%m/%Y')

In [ ]:
#создадим поле с грейдом из названия вакансии
da['grade'] = ''

for i, title in enumerate(da['name'].tolist()):
    if 'junior' in str(title):
        da.loc[i, 'grade'] = 'junior'
    elif 'middle' in str(title):
        da.loc[i, 'grade'] = 'middle'
    elif 'senior' in str(title):
        da.loc[i, 'grade'] = 'senior'
    else:
      da.loc[i, 'grade'] = np.NaN

In [ ]:
#не везде совпадают колонка опыт и грейд. Пустые ячейки в grade заполним значениями из experience
da['grade'] = da['grade'].T.fillna(da['experience']).T
da['grade'] = da['grade'].apply(lambda row: row.split(' ')[0])
da['grade'] = da['grade'].apply(lambda x: x.lower())

In [ ]:
#создадим поле только с требованием опыта в годах
da['experience_year'] = da['experience'].apply(lambda row: row.split('years')[0])
da['experience_year'] = da['experience_year'].apply(lambda row: row.split('(')[1])
da['experience_year'] = da['experience_year'].str.replace(')','')

<ipython-input-13-fdc19e6ed975>:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  da['experience_year'] = da['experience_year'].str.replace(')','')


В результате предобработки данных датасета по вакансиям Аналитик данных были выполнены следубщие задачи:
1. Удалены данные, которые не пригодятся для исследования.
2. Изучены числовые показатели, аномалий не найдено.
3. Проведена проверка на явные и неявные дубликаты.
4. Преобразован формат некоторых данных для дальнейшей работы с ними.
5. Добавлены столбцы: месяц публикации вакансии, грейд (сделан на основе названия вакансии или требований к опыту).

## Анализ данных DA

### Анализ скиллов DA

#### Скиллы DA

Обработаем данные по скиллам

In [ ]:
#сделаем в рабочем поле с ключевыми навыками нижний регистр, преобразуем в список и уберём ненужные элементы
da['key_skills_process'] = da['key_skills'].str.lower()
da['key_skills_process'] = da['key_skills_process'].str.split(', ')
da['key_skills_process'] = [[element.replace("[", "") for element in sublist] for sublist in da['key_skills_process']]
da['key_skills_process'] = [[element.replace("'", "") for element in sublist] for sublist in da['key_skills_process']]
da['key_skills_process'] = [[element.replace("]", "") for element in sublist] for sublist in da['key_skills_process']]

In [ ]:
#создадим пустой словарь, куда будут добавляться значения из поля 'key_skills_process' в зависимости от названия скилла
skill_count = {}

for row in da['key_skills_process']:
    for element in row:
        if element in skill_count:
            skill_count[element] += 1
        else:
            skill_count[element] = 1

print(skill_count)

{'ms sql': 35, 'power pivot': 2, 'прогнозирование': 11, 'sql': 436, 'olap (online analytical processing)': 2, 'коммуникация': 104, 'анализ данных': 139, 'ms powerpoint': 18, 'power bi': 91, 'power query': 2, 'конкурентная аналитика': 1, 'vba': 9, 'яндекс.директ': 1, 'маркетинговый анализ': 3, 'контекстная реклама': 1, 'web аналитика': 7, 'аналитическое мышление': 122, 'лидогенерация': 1, 'веб-аналитика': 2, 'roi': 6, 'google analytics': 11, 'яндекс.метрика': 10, '': 129, 'управленческая отчетность': 3, 'olap': 2, 'sql запросы': 3, 'excel': 6, 'confluence': 59, 'документация': 113, 'jira': 4, 'python': 377, 'data analysis': 171, 'умение работать в режиме многозадачности': 1, 'работа с большим объемом информации': 38, 'google таблицы': 4, 'отчеты': 1, 'bi': 20, 'hadoop': 23, 'математическая статистика': 60, 'http': 1, 'pandas': 204, 'поиск аномалий': 1, 'clickhouse': 48, 'linux': 10, 'анализ временных рядов': 1, 'git': 14, 'аналитический склад ума': 1, 'ms excel': 42, 'подготовка презент

In [ ]:
#посмотрим в сыром виде, какие скиллы наиболее часто встречаются
skills_raw = pd.DataFrame.from_dict(skill_count, orient='index', columns=['count']).sort_values(by='count', ascending=False)
skills_raw.head()

,count
sql,436
python,377
pandas,204
data analysis,171
анализ данных,139


In [ ]:
#укрупним группы. За основу возьмём наиболее часто встречающиеся элементы
skill_groups = {'sql (hard)':[], 'python (hard)':[], 'data analysis (hard)':[], 'документация (soft)': [], 'коммуникация (soft)':[],
                 'powerbi (hard)':[], 'tableau (hard)':[], 'a/b тесты (hard)':[], 'статистика (hard)':[], 'confluence (hard)':[],
                'clickhouse (hard)':[], 'юнит-экономика (hard)':[], 'excel (hard)':[], 'базы данных (hard)':[],
                'работа с большим объемом информации (soft)':[], 'etl (hard)':[], 'английский язык (hard)':[],
                 'airflow (hard)':[],'spark (hard)':[], 'hadoop (hard)':[], 'business intelligence (hard)':[],
                'machine learning (hard)': [], 'презентация  (soft)':[],'google analytics (hard)':[], 'oracle (hard)':[], 'powerpoint (hard)':[],
                'активность  (soft)':[], 'superset (hard)':[],'visualisation (hard)':[], 'работа в команде  (soft)':[],
                'git (hard)':[], 'критическое мышление  (soft)':[], 'яндекс.метрика (hard)':[], 'linux (hard)':[],
                'принятие решений  (soft)':[],'grafana (hard)':[], 'vba (hard)':[], 'api (hard)':[], 'внимательность (soft)':[],
                'greenplum (hard)':[], 'pyspark (hard)':[], 'elasticsearch (hard)':[], 'google docs (hard)':[],
                'data scince (hard)':[], 'hive (hard)':[], 'оценка рисков (hard)':[], 'креативность (soft)':[],
                'бизнес-показатели (hard)':[], 'product management (hard)':[], 'управление  (soft)':[], 'маркетинг':[], 'supply (hard)':[], 'other':[]}

In [ ]:
#по ключевым словам объединим скиллы по группам
for skill, count in skill_count.items():
    for _ in range(count):
        if 'sql' in skill:
            skill_groups['sql (hard)'].append(skill)
        elif 'python' in skill or 'pandas' in skill or 'питон' in skill or 'scipy' in skill or 'plotly' in skill or 'seborn' in skill or 'matplotlib' in skill:
            skill_groups['python (hard)'].append(skill)
        elif 'analy' in skill or 'анали' in skill or 'гипот' in skill or 'olap' in skill:
            skill_groups['data analysis (hard)'].append(skill)
        elif 'документ' in skill or 'отчет' in skill:
            skill_groups['документация (soft)'].append(skill)
        elif 'комму' in skill or 'речь' in skill or 'пис' in skill or 'грамот' in skill:
            skill_groups['коммуникация (soft)'].append(skill)
        elif 'power' in skill and 'bi' in skill:
            skill_groups['powerbi (hard)'].append(skill)
        elif 'tableau' in skill:
            skill_groups['tableau (hard)'].append(skill)
        elif 'тест' in skill and 'a' in skill:
            skill_groups['a/b тесты (hard)'].append(skill)
        elif 'статис' in skill or 'statis' in skill:
            skill_groups['статистика (hard)'].append(skill)
        elif 'confl' in skill:
            skill_groups['confluence (hard)'].append(skill)
        elif 'clickh' in skill:
            skill_groups['clickhouse (hard)'].append(skill)
        elif 'юнит' in skill or 'unit' in skill:
            skill_groups['юнит-экономика (hard)'].append(skill)
        elif 'exc' in skill:
            skill_groups['excel (hard)'].append(skill)
        elif 'баз' in skill or 'субд' in skill or 'base' in skill:
            skill_groups['базы данных (hard)'].append(skill)
        elif 'big data' in skill or 'больш' in skill:
            skill_groups['работа с большим объемом информации (soft)'].append(skill)
        elif 'etl' in skill:
            skill_groups['etl (hard)'].append(skill)
        elif 'англ' in skill or 'engl' in skill:
            skill_groups['английский язык (hard)'].append(skill)
        elif 'airflow' in skill:
            skill_groups['airflow (hard)'].append(skill)
        elif 'spark' in skill:
            skill_groups['spark (hard)'].append(skill)
        elif 'hadoop' in skill:
            skill_groups['hadoop (hard)'].append(skill)
        elif 'intelli' in skill or 'bi' in skill or 'sap' in skill:
            skill_groups['business intelligence (hard)'].append(skill)
        elif 'learning' in skill or 'машин' in skill or skill=='ml':
            skill_groups['machine learning (hard)'].append(skill)
        elif 'презент' in skill:
            skill_groups['презентация  (soft)'].append(skill)
        elif 'goog' in skill and 'anal' in skill:
            skill_groups['google analytics (hard)'].append(skill)
        elif 'oracle' in skill:
            skill_groups['oracle (hard)'].append(skill)
        elif 'power' in skill and 'point' in skill:
            skill_groups['powerpoint (hard)'].append(skill)
        elif 'активн' in skill:
            skill_groups['активность  (soft)'].append(skill)
        elif 'superset' in skill:
            skill_groups['superset (hard)'].append(skill)
        elif 'visua' in skill or 'визуа' in skill:
            skill_groups['visualisation (hard)'].append(skill)
        elif 'команд' in skill in skill:
            skill_groups['работа в команде  (soft)'].append(skill)
        elif 'git' in skill:
            skill_groups['git (hard)'].append(skill)
        elif 'крити' in skill:
            skill_groups['критическое мышление  (soft)'].append(skill)
        elif 'метр' in skill and 'янд' in skill or 'yan' in skill:
            skill_groups['яндекс.метрика (hard)'].append(skill)
        elif 'linux' in skill:
            skill_groups['linux (hard)'].append(skill)
        elif 'реш' in skill:
            skill_groups['принятие решений  (soft)'].append(skill)
        elif 'grafana' in skill:
            skill_groups['grafana (hard)'].append(skill)
        elif 'vba' in skill:
            skill_groups['vba (hard)'].append(skill)
        elif 'api' in skill:
            skill_groups['api (hard)'].append(skill)
        elif 'внима' in skill:
            skill_groups['внимательность (soft)'].append(skill)
        elif 'greenplum' in skill:
            skill_groups['greenplum (hard)'].append(skill)
        elif 'pyspark' in skill:
            skill_groups['pyspark (hard)'].append(skill)
        elif 'elasticsearch' in skill:
            skill_groups['elasticsearch (hard)'].append(skill)
        elif 'google' in skill:
            skill_groups['google docs (hard)'].append(skill)
        elif 'data' in skill and 'sci' in skill or 'nlp' in skill:
            skill_groups['data scince (hard)'].append(skill)
        elif 'hive' in skill:
            skill_groups['hive (hard)'].append(skill)
        elif 'риск' in skill or 'risk' in skill:
            skill_groups['оценка рисков (hard)'].append(skill)
        elif 'креа' in skill:
            skill_groups['креативность (soft)'].append(skill)
        elif 'показ' in skill or 'roi' in skill or 'cpl' in skill or 'cpa' in skill or 'cor' in skill or 'ltv' in skill:
            skill_groups['бизнес-показатели (hard)'].append(skill)
        elif 'продукт' in skill or 'product' in skill:
            skill_groups['product management (hard)'].append(skill)
        elif 'управл' in skill or 'лидер' in skill or 'руков' in skill or 'руков' in skill or 'организа' in skill or 'managem' in skill:
            skill_groups['управление  (soft)'].append(skill)
        elif 'маркетинг' in skill or 'marketi' in skill or 'smm' in skill or 'rtb' in skill or 'реклам' in skill or 'лидо' in skill:
            skill_groups['маркетинг'].append(skill)
        elif 'supply' in skill or 'запас' in skill or 'demand' in skill or 'складс' in skill:
            skill_groups['supply (hard)'].append(skill)
        else:
            skill_groups["other"].append(skill)


skills_ad_df = pd.DataFrame({"skill": list(skill_groups.keys()), "count": [len(value) for value in skill_groups.values()]})
skills_ad_df = skills_ad_df.sort_values(by='skill').reset_index(drop=True)
skills_ad_df.head()

,skill,count
0,a/b тесты (hard),66
1,airflow (hard),31
2,api (hard),15
3,business intelligence (hard),41
4,clickhouse (hard),48


#### Скиллы с разбивкой на грейды

Проделаем то же самое отдельно по каждому грейду.

##### Junior

In [ ]:
skill_jr_count = {}

for index, row in da.iterrows():
    if row['grade'] == 'junior':
        for skill in row['key_skills_process']:
            if skill in skill_jr_count:
                skill_jr_count[skill] += 1
            else:
                skill_jr_count[skill] = 1

In [ ]:
skill_jr_groups = {'sql (hard)':[], 'python (hard)':[], 'data analysis (hard)':[], 'документация (soft)': [], 'коммуникация (soft)':[],
                 'powerbi (hard)':[], 'tableau (hard)':[], 'a/b тесты (hard)':[], 'статистика (hard)':[], 'confluence (hard)':[],
                'clickhouse (hard)':[], 'юнит-экономика (hard)':[], 'excel (hard)':[], 'базы данных (hard)':[],
                'работа с большим объемом информации (soft)':[], 'etl (hard)':[], 'английский язык (hard)':[],
                 'airflow (hard)':[],'spark (hard)':[], 'hadoop (hard)':[], 'business intelligence (hard)':[],
                'machine learning (hard)': [], 'презентация  (soft)':[],'google analytics (hard)':[], 'oracle (hard)':[], 'powerpoint (hard)':[],
                'активность  (soft)':[], 'superset (hard)':[],'visualisation (hard)':[], 'работа в команде  (soft)':[],
                'git (hard)':[], 'критическое мышление  (soft)':[], 'яндекс.метрика (hard)':[], 'linux (hard)':[],
                'принятие решений  (soft)':[],'grafana (hard)':[], 'vba (hard)':[], 'api (hard)':[], 'внимательность (soft)':[],
                'greenplum (hard)':[], 'pyspark (hard)':[], 'elasticsearch (hard)':[], 'google docs (hard)':[],
                'data scince (hard)':[], 'hive (hard)':[], 'оценка рисков (hard)':[], 'креативность (soft)':[],
                'бизнес-показатели (hard)':[], 'product management (hard)':[], 'управление  (soft)':[], 'маркетинг':[], 'supply (hard)':[], 'other':[]}

In [ ]:
for skill, count in skill_jr_count.items():
    for _ in range(count):
        if 'sql' in skill:
            skill_jr_groups['sql (hard)'].append(skill)
        elif 'python' in skill or 'pandas' in skill or 'питон' in skill or 'scipy' in skill or 'plotly' in skill or 'seborn' in skill or 'matplotlib' in skill:
            skill_jr_groups['python (hard)'].append(skill)
        elif 'analy' in skill or 'анали' in skill or 'гипот' in skill or 'olap' in skill:
            skill_jr_groups['data analysis (hard)'].append(skill)
        elif 'документ' in skill or 'отчет' in skill:
            skill_jr_groups['документация (soft)'].append(skill)
        elif 'комму' in skill or 'речь' in skill or 'пис' in skill or 'грамот' in skill:
            skill_jr_groups['коммуникация (soft)'].append(skill)
        elif 'power' in skill and 'bi' in skill:
            skill_jr_groups['powerbi (hard)'].append(skill)
        elif 'tableau' in skill:
            skill_jr_groups['tableau (hard)'].append(skill)
        elif 'тест' in skill and 'a' in skill:
            skill_jr_groups['a/b тесты (hard)'].append(skill)
        elif 'статис' in skill or 'statis' in skill:
            skill_jr_groups['статистика (hard)'].append(skill)
        elif 'confl' in skill:
            skill_jr_groups['confluence (hard)'].append(skill)
        elif 'clickh' in skill:
            skill_jr_groups['clickhouse (hard)'].append(skill)
        elif 'юнит' in skill or 'unit' in skill:
            skill_jr_groups['юнит-экономика (hard)'].append(skill)
        elif 'exc' in skill:
            skill_jr_groups['excel (hard)'].append(skill)
        elif 'баз' in skill or 'субд' in skill or 'base' in skill:
            skill_jr_groups['базы данных (hard)'].append(skill)
        elif 'big data' in skill or 'больш' in skill:
            skill_jr_groups['работа с большим объемом информации (soft)'].append(skill)
        elif 'etl' in skill:
            skill_jr_groups['etl (hard)'].append(skill)
        elif 'англ' in skill or 'engl' in skill:
            skill_jr_groups['английский язык (hard)'].append(skill)
        elif 'airflow' in skill:
            skill_jr_groups['airflow (hard)'].append(skill)
        elif 'spark' in skill:
            skill_jr_groups['spark (hard)'].append(skill)
        elif 'hadoop' in skill:
            skill_jr_groups['hadoop (hard)'].append(skill)
        elif 'intelli' in skill or 'bi' in skill or 'sap' in skill:
            skill_jr_groups['business intelligence (hard)'].append(skill)
        elif 'learning' in skill or 'машин' in skill or skill=='ml':
            skill_jr_groups['machine learning (hard)'].append(skill)
        elif 'презент' in skill:
            skill_jr_groups['презентация  (soft)'].append(skill)
        elif 'goog' in skill and 'anal' in skill:
            skill_jr_groups['google analytics (hard)'].append(skill)
        elif 'oracle' in skill:
            skill_jr_groups['oracle (hard)'].append(skill)
        elif 'power' in skill and 'point' in skill:
            skill_jr_groups['powerpoint (hard)'].append(skill)
        elif 'активн' in skill:
            skill_jr_groups['активность  (soft)'].append(skill)
        elif 'superset' in skill:
            skill_jr_groups['superset (hard)'].append(skill)
        elif 'visua' in skill or 'визуа' in skill:
            skill_jr_groups['visualisation (hard)'].append(skill)
        elif 'команд' in skill:
            skill_jr_groups['работа в команде  (soft)'].append(skill)
        elif 'git' in skill:
            skill_jr_groups['git (hard)'].append(skill)
        elif 'крити' in skill:
            skill_jr_groups['критическое мышление  (soft)'].append(skill)
        elif 'метр' in skill and 'янд' in skill or 'yan' in skill:
            skill_jr_groups['яндекс.метрика (hard)'].append(skill)
        elif 'linux' in skill:
            skill_jr_groups['linux (hard)'].append(skill)
        elif 'реш' in skill:
            skill_jr_groups['принятие решений  (soft)'].append(skill)
        elif 'grafana' in skill:
            skill_jr_groups['grafana (hard)'].append(skill)
        elif 'vba' in skill:
            skill_jr_groups['vba (hard)'].append(skill)
        elif 'api' in skill:
            skill_jr_groups['api (hard)'].append(skill)
        elif 'внима' in skill:
            skill_jr_groups['внимательность (soft)'].append(skill)
        elif 'greenplum' in skill:
            skill_jr_groups['greenplum (hard)'].append(skill)
        elif 'pyspark' in skill:
            skill_jr_groups['pyspark (hard)'].append(skill)
        elif 'elasticsearch' in skill:
            skill_jr_groups['elasticsearch (hard)'].append(skill)
        elif 'google' in skill:
            skill_jr_groups['google docs (hard)'].append(skill)
        elif 'data' in skill and 'sci' in skill or 'nlp' in skill:
            skill_jr_groups['data scince (hard)'].append(skill)
        elif 'hive' in skill:
            skill_jr_groups['hive (hard)'].append(skill)
        elif 'риск' in skill or 'risk' in skill:
            skill_jr_groups['оценка рисков (hard)'].append(skill)
        elif 'креа' in skill:
            skill_jr_groups['креативность (soft)'].append(skill)
        elif 'показ' in skill or 'roi' in skill or 'cpl' in skill or 'cpa' in skill or 'cor' in skill or 'ltv' in skill:
            skill_jr_groups['бизнес-показатели (hard)'].append(skill)
        elif 'продукт' in skill or 'product' in skill:
            skill_jr_groups['product management (hard)'].append(skill)
        elif 'управл' in skill or 'лидер' in skill or 'руков' in skill or 'руков' in skill or 'организа' in skill or 'managem' in skill:
            skill_jr_groups['управление  (soft)'].append(skill)
        elif 'маркетинг' in skill or 'marketi' in skill or 'smm' in skill or 'rtb' in skill or 'реклам' in skill or 'лидо' in skill:
            skill_jr_groups['маркетинг'].append(skill)
        elif 'supply' in skill or 'запас' in skill or 'demand' in skill or 'складс' in skill:
            skill_jr_groups['supply (hard)'].append(skill)
        else:
            skill_jr_groups["other"].append(skill)


skills_ad_jr_df = pd.DataFrame({"skill": list(skill_jr_groups.keys()), "count_jrp": [len(value) for value in skill_jr_groups.values()]})
skills_ad_jr_df = skills_ad_jr_df.sort_values(by='skill').reset_index(drop=True)
skills_ad_jr_df.head()

,skill,count_jrp
0,a/b тесты (hard),0
1,airflow (hard),0
2,api (hard),0
3,business intelligence (hard),1
4,clickhouse (hard),0


##### Junior+

In [ ]:
skill_jrp_count = {}

for index, row in da.iterrows():
    if row['grade'] == 'junior+':
        for skill in row['key_skills_process']:
            if skill in skill_jrp_count:
                skill_jrp_count[skill] += 1
            else:
                skill_jrp_count[skill] = 1

In [ ]:
#укрупним группы. За основу возьмём наиболее часто встречающиеся элементы
skill_jrp_groups = {'sql (hard)':[], 'python (hard)':[], 'data analysis (hard)':[], 'документация (soft)': [], 'коммуникация (soft)':[],
                 'powerbi (hard)':[], 'tableau (hard)':[], 'a/b тесты (hard)':[], 'статистика (hard)':[], 'confluence (hard)':[],
                'clickhouse (hard)':[], 'юнит-экономика (hard)':[], 'excel (hard)':[], 'базы данных (hard)':[],
                'работа с большим объемом информации (soft)':[], 'etl (hard)':[], 'английский язык (hard)':[],
                 'airflow (hard)':[],'spark (hard)':[], 'hadoop (hard)':[], 'business intelligence (hard)':[],
                'machine learning (hard)': [], 'презентация  (soft)':[],'google analytics (hard)':[], 'oracle (hard)':[], 'powerpoint (hard)':[],
                'активность  (soft)':[], 'superset (hard)':[],'visualisation (hard)':[], 'работа в команде  (soft)':[],
                'git (hard)':[], 'критическое мышление  (soft)':[], 'яндекс.метрика (hard)':[], 'linux (hard)':[],
                'принятие решений  (soft)':[],'grafana (hard)':[], 'vba (hard)':[], 'api (hard)':[], 'внимательность (soft)':[],
                'greenplum (hard)':[], 'pyspark (hard)':[], 'elasticsearch (hard)':[], 'google docs (hard)':[],
                'data scince (hard)':[], 'hive (hard)':[], 'оценка рисков (hard)':[], 'креативность (soft)':[],
                'бизнес-показатели (hard)':[], 'product management (hard)':[], 'управление  (soft)':[], 'маркетинг':[], 'supply (hard)':[], 'other':[]}

In [ ]:
for skill, count in skill_jrp_count.items():
    for _ in range(count):
        if 'sql' in skill:
            skill_jrp_groups['sql (hard)'].append(skill)
        elif 'python' in skill or 'pandas' in skill or 'питон' in skill or 'scipy' in skill or 'plotly' in skill or 'seborn' in skill or 'matplotlib' in skill:
            skill_jrp_groups['python (hard)'].append(skill)
        elif 'analy' in skill or 'анали' in skill or 'гипот' in skill or 'olap' in skill:
            skill_jrp_groups['data analysis (hard)'].append(skill)
        elif 'документ' in skill or 'отчет' in skill:
            skill_jrp_groups['документация (soft)'].append(skill)
        elif 'комму' in skill or 'речь' in skill or 'пис' in skill or 'грамот' in skill:
            skill_jrp_groups['коммуникация (soft)'].append(skill)
        elif 'power' in skill and 'bi' in skill:
            skill_jrp_groups['powerbi (hard)'].append(skill)
        elif 'tableau' in skill:
            skill_jrp_groups['tableau (hard)'].append(skill)
        elif 'тест' in skill and 'a' in skill:
            skill_jrp_groups['a/b тесты (hard)'].append(skill)
        elif 'статис' in skill or 'statis' in skill:
            skill_jrp_groups['статистика (hard)'].append(skill)
        elif 'confl' in skill:
            skill_jrp_groups['confluence (hard)'].append(skill)
        elif 'clickh' in skill:
            skill_jrp_groups['clickhouse (hard)'].append(skill)
        elif 'юнит' in skill or 'unit' in skill:
            skill_jrp_groups['юнит-экономика (hard)'].append(skill)
        elif 'exc' in skill:
            skill_jrp_groups['excel (hard)'].append(skill)
        elif 'баз' in skill or 'субд' in skill or 'base' in skill:
            skill_jrp_groups['базы данных (hard)'].append(skill)
        elif 'big data' in skill or 'больш' in skill:
            skill_jrp_groups['работа с большим объемом информации (soft)'].append(skill)
        elif 'etl' in skill:
            skill_jrp_groups['etl (hard)'].append(skill)
        elif 'англ' in skill or 'engl' in skill:
            skill_jrp_groups['английский язык (hard)'].append(skill)
        elif 'airflow' in skill:
            skill_jrp_groups['airflow (hard)'].append(skill)
        elif 'spark' in skill:
            skill_jrp_groups['spark (hard)'].append(skill)
        elif 'hadoop' in skill:
            skill_jrp_groups['hadoop (hard)'].append(skill)
        elif 'intelli' in skill or 'bi' in skill or 'sap' in skill:
            skill_jrp_groups['business intelligence (hard)'].append(skill)
        elif 'learning' in skill or 'машин' in skill or skill=='ml':
            skill_jrp_groups['machine learning (hard)'].append(skill)
        elif 'презент' in skill:
            skill_jrp_groups['презентация  (soft)'].append(skill)
        elif 'goog' in skill and 'anal' in skill:
            skill_jrp_groups['google analytics (hard)'].append(skill)
        elif 'oracle' in skill:
            skill_jrp_groups['oracle (hard)'].append(skill)
        elif 'power' in skill and 'point' in skill:
            skill_jrp_groups['powerpoint (hard)'].append(skill)
        elif 'активн' in skill:
            skill_jrp_groups['активность  (soft)'].append(skill)
        elif 'superset' in skill:
            skill_jrp_groups['superset (hard)'].append(skill)
        elif 'visua' in skill or 'визуа' in skill:
            skill_jrp_groups['visualisation (hard)'].append(skill)
        elif 'команд' in skill:
            skill_jrp_groups['работа в команде  (soft)'].append(skill)
        elif 'git' in skill:
            skill_jrp_groups['git (hard)'].append(skill)
        elif 'крити' in skill:
            skill_jrp_groups['критическое мышление  (soft)'].append(skill)
        elif 'метр' in skill and 'янд' in skill or 'yan' in skill:
            skill_jrp_groups['яндекс.метрика (hard)'].append(skill)
        elif 'linux' in skill:
            skill_jrp_groups['linux (hard)'].append(skill)
        elif 'реш' in skill:
            skill_jrp_groups['принятие решений  (soft)'].append(skill)
        elif 'grafana' in skill:
            skill_jrp_groups['grafana (hard)'].append(skill)
        elif 'vba' in skill:
            skill_jrp_groups['vba (hard)'].append(skill)
        elif 'api' in skill:
            skill_jrp_groups['api (hard)'].append(skill)
        elif 'внима' in skill:
            skill_jrp_groups['внимательность (soft)'].append(skill)
        elif 'greenplum' in skill:
            skill_jrp_groups['greenplum (hard)'].append(skill)
        elif 'pyspark' in skill:
            skill_jrp_groups['pyspark (hard)'].append(skill)
        elif 'elasticsearch' in skill:
            skill_jrp_groups['elasticsearch (hard)'].append(skill)
        elif 'google' in skill:
            skill_jrp_groups['google docs (hard)'].append(skill)
        elif 'data' in skill and 'sci' in skill or 'nlp' in skill:
            skill_jrp_groups['data scince (hard)'].append(skill)
        elif 'hive' in skill:
            skill_jrp_groups['hive (hard)'].append(skill)
        elif 'риск' in skill or 'risk' in skill:
            skill_jrp_groups['оценка рисков (hard)'].append(skill)
        elif 'креа' in skill:
            skill_jrp_groups['креативность (soft)'].append(skill)
        elif 'показ' in skill or 'roi' in skill or 'cpl' in skill or 'cpa' in skill or 'cor' in skill or 'ltv' in skill:
            skill_jrp_groups['бизнес-показатели (hard)'].append(skill)
        elif 'продукт' in skill or 'product' in skill:
            skill_jrp_groups['product management (hard)'].append(skill)
        elif 'управл' in skill or 'лидер' in skill or 'руков' in skill or 'руков' in skill or 'организа' in skill or 'managem' in skill:
            skill_jrp_groups['управление  (soft)'].append(skill)
        elif 'маркетинг' in skill or 'marketi' in skill or 'smm' in skill or 'rtb' in skill or 'реклам' in skill or 'лидо' in skill:
            skill_jrp_groups['маркетинг'].append(skill)
        elif 'supply' in skill or 'запас' in skill or 'demand' in skill or 'складс' in skill:
            skill_jrp_groups['supply (hard)'].append(skill)
        else:
            skill_jrp_groups["other"].append(skill)


skills_ad_jrp_df = pd.DataFrame({"skill": list(skill_jrp_groups.keys()), "count_jrp": [len(value) for value in skill_jrp_groups.values()]})
skills_ad_jrp_df = skills_ad_jrp_df.sort_values(by='skill').reset_index(drop=True)
skills_ad_jrp_df.head()

,skill,count_jrp
0,a/b тесты (hard),27
1,airflow (hard),8
2,api (hard),5
3,business intelligence (hard),28
4,clickhouse (hard),19


##### Middle

In [ ]:
skill_mid_count = {}

for index, row in da.iterrows():
    if row['grade'] == 'middle':
        for skill in row['key_skills_process']:
            if skill in skill_mid_count:
                skill_mid_count[skill] += 1
            else:
                skill_mid_count[skill] = 1

In [ ]:
#укрупним группы. За основу возьмём наиболее часто встречающиеся элементы
skill_mid_groups = {'sql (hard)':[], 'python (hard)':[], 'data analysis (hard)':[], 'документация (soft)': [], 'коммуникация (soft)':[],
                 'powerbi (hard)':[], 'tableau (hard)':[], 'a/b тесты (hard)':[], 'статистика (hard)':[], 'confluence (hard)':[],
                'clickhouse (hard)':[], 'юнит-экономика (hard)':[], 'excel (hard)':[], 'базы данных (hard)':[],
                'работа с большим объемом информации (soft)':[], 'etl (hard)':[], 'английский язык (hard)':[],
                 'airflow (hard)':[],'spark (hard)':[], 'hadoop (hard)':[], 'business intelligence (hard)':[],
                'machine learning (hard)': [], 'презентация  (soft)':[],'google analytics (hard)':[], 'oracle (hard)':[], 'powerpoint (hard)':[],
                'активность  (soft)':[], 'superset (hard)':[],'visualisation (hard)':[], 'работа в команде  (soft)':[],
                'git (hard)':[], 'критическое мышление  (soft)':[], 'яндекс.метрика (hard)':[], 'linux (hard)':[],
                'принятие решений  (soft)':[],'grafana (hard)':[], 'vba (hard)':[], 'api (hard)':[], 'внимательность (soft)':[],
                'greenplum (hard)':[], 'pyspark (hard)':[], 'elasticsearch (hard)':[], 'google docs (hard)':[],
                'data scince (hard)':[], 'hive (hard)':[], 'оценка рисков (hard)':[], 'креативность (soft)':[],
                'бизнес-показатели (hard)':[], 'product management (hard)':[], 'управление  (soft)':[], 'маркетинг':[], 'supply (hard)':[], 'other':[]}

In [ ]:
for skill, count in skill_mid_count.items():
    for _ in range(count):
        if 'sql' in skill:
            skill_mid_groups['sql (hard)'].append(skill)
        elif 'python' in skill or 'pandas' in skill or 'питон' in skill or 'scipy' in skill or 'plotly' in skill or 'seborn' in skill or 'matplotlib' in skill:
            skill_mid_groups['python (hard)'].append(skill)
        elif 'analy' in skill or 'анали' in skill or 'гипот' in skill or 'olap' in skill:
            skill_mid_groups['data analysis (hard)'].append(skill)
        elif 'документ' in skill or 'отчет' in skill:
            skill_mid_groups['документация (soft)'].append(skill)
        elif 'комму' in skill or 'речь' in skill or 'пис' in skill or 'грамот' in skill:
            skill_mid_groups['коммуникация (soft)'].append(skill)
        elif 'power' in skill and 'bi' in skill:
            skill_mid_groups['powerbi (hard)'].append(skill)
        elif 'tableau' in skill:
            skill_mid_groups['tableau (hard)'].append(skill)
        elif 'тест' in skill and 'a' in skill:
            skill_mid_groups['a/b тесты (hard)'].append(skill)
        elif 'статис' in skill or 'statis' in skill:
            skill_mid_groups['статистика (hard)'].append(skill)
        elif 'confl' in skill:
            skill_mid_groups['confluence (hard)'].append(skill)
        elif 'clickh' in skill:
            skill_mid_groups['clickhouse (hard)'].append(skill)
        elif 'юнит' in skill or 'unit' in skill:
            skill_mid_groups['юнит-экономика (hard)'].append(skill)
        elif 'exc' in skill:
            skill_mid_groups['excel (hard)'].append(skill)
        elif 'баз' in skill or 'субд' in skill or 'base' in skill:
            skill_mid_groups['базы данных (hard)'].append(skill)
        elif 'big data' in skill or 'больш' in skill:
            skill_mid_groups['работа с большим объемом информации (soft)'].append(skill)
        elif 'etl' in skill:
            skill_mid_groups['etl (hard)'].append(skill)
        elif 'англ' in skill or 'engl' in skill:
            skill_mid_groups['английский язык (hard)'].append(skill)
        elif 'airflow' in skill:
            skill_mid_groups['airflow (hard)'].append(skill)
        elif 'spark' in skill:
            skill_mid_groups['spark (hard)'].append(skill)
        elif 'hadoop' in skill:
            skill_mid_groups['hadoop (hard)'].append(skill)
        elif 'intelli' in skill or 'bi' in skill or 'sap' in skill:
            skill_mid_groups['business intelligence (hard)'].append(skill)
        elif 'learning' in skill or 'машин' in skill or skill=='ml':
            skill_mid_groups['machine learning (hard)'].append(skill)
        elif 'презент' in skill:
            skill_mid_groups['презентация  (soft)'].append(skill)
        elif 'goog' in skill and 'anal' in skill:
            skill_mid_groups['google analytics (hard)'].append(skill)
        elif 'oracle' in skill:
            skill_mid_groups['oracle (hard)'].append(skill)
        elif 'power' in skill and 'point' in skill:
            skill_mid_groups['powerpoint (hard)'].append(skill)
        elif 'активн' in skill:
            skill_mid_groups['активность  (soft)'].append(skill)
        elif 'superset' in skill:
            skill_mid_groups['superset (hard)'].append(skill)
        elif 'visua' in skill or 'визуа' in skill:
            skill_mid_groups['visualisation (hard)'].append(skill)
        elif 'команд' in skill:
            skill_mid_groups['работа в команде  (soft)'].append(skill)
        elif 'git' in skill:
            skill_mid_groups['git (hard)'].append(skill)
        elif 'крити' in skill:
            skill_mid_groups['критическое мышление  (soft)'].append(skill)
        elif 'метр' in skill and 'янд' in skill or 'yan' in skill:
            skill_mid_groups['яндекс.метрика (hard)'].append(skill)
        elif 'linux' in skill:
            skill_mid_groups['linux (hard)'].append(skill)
        elif 'реш' in skill:
            skill_mid_groups['принятие решений  (soft)'].append(skill)
        elif 'grafana' in skill:
            skill_mid_groups['grafana (hard)'].append(skill)
        elif 'vba' in skill:
            skill_mid_groups['vba (hard)'].append(skill)
        elif 'api' in skill:
            skill_mid_groups['api (hard)'].append(skill)
        elif 'внима' in skill:
            skill_mid_groups['внимательность (soft)'].append(skill)
        elif 'greenplum' in skill:
            skill_mid_groups['greenplum (hard)'].append(skill)
        elif 'pyspark' in skill:
            skill_mid_groups['pyspark (hard)'].append(skill)
        elif 'elasticsearch' in skill:
            skill_mid_groups['elasticsearch (hard)'].append(skill)
        elif 'google' in skill:
            skill_mid_groups['google docs (hard)'].append(skill)
        elif 'data' in skill and 'sci' in skill or 'nlp' in skill:
            skill_mid_groups['data scince (hard)'].append(skill)
        elif 'hive' in skill:
            skill_mid_groups['hive (hard)'].append(skill)
        elif 'риск' in skill or 'risk' in skill:
            skill_mid_groups['оценка рисков (hard)'].append(skill)
        elif 'креа' in skill:
            skill_mid_groups['креативность (soft)'].append(skill)
        elif 'показ' in skill or 'roi' in skill or 'cpl' in skill or 'cpa' in skill or 'cor' in skill or 'ltv' in skill:
            skill_mid_groups['бизнес-показатели (hard)'].append(skill)
        elif 'продукт' in skill or 'product' in skill:
            skill_mid_groups['product management (hard)'].append(skill)
        elif 'управл' in skill or 'лидер' in skill or 'руков' in skill or 'руков' in skill or 'организа' in skill or 'managem' in skill:
            skill_mid_groups['управление  (soft)'].append(skill)
        elif 'маркетинг' in skill or 'marketi' in skill or 'smm' in skill or 'rtb' in skill or 'реклам' in skill or 'лидо' in skill:
            skill_mid_groups['маркетинг'].append(skill)
        elif 'supply' in skill or 'запас' in skill or 'demand' in skill or 'складс' in skill:
            skill_mid_groups['supply (hard)'].append(skill)
        else:
            skill_mid_groups["other"].append(skill)


skills_ad_mid_df = pd.DataFrame({"skill": list(skill_mid_groups.keys()), "count_mid": [len(value) for value in skill_mid_groups.values()]})
skills_ad_mid_df = skills_ad_mid_df.sort_values(by='skill').reset_index(drop=True)
skills_ad_mid_df.head()

,skill,count_mid
0,a/b тесты (hard),35
1,airflow (hard),22
2,api (hard),6
3,business intelligence (hard),11
4,clickhouse (hard),28


##### Senior

In [ ]:
skill_sen_count = {}

for index, row in da.iterrows():
    if row['grade'] == 'senior':
        for skill in row['key_skills_process']:
            if skill in skill_sen_count:
                skill_sen_count[skill] += 1
            else:
                skill_sen_count[skill] = 1

In [ ]:
#укрупним группы. За основу возьмём наиболее часто встречающиеся элементы
skill_sen_groups = {'sql (hard)':[], 'python (hard)':[], 'data analysis (hard)':[], 'документация (soft)': [], 'коммуникация (soft)':[],
                 'powerbi (hard)':[], 'tableau (hard)':[], 'a/b тесты (hard)':[], 'статистика (hard)':[], 'confluence (hard)':[],
                'clickhouse (hard)':[], 'юнит-экономика (hard)':[], 'excel (hard)':[], 'базы данных (hard)':[],
                'работа с большим объемом информации (soft)':[], 'etl (hard)':[], 'английский язык (hard)':[],
                 'airflow (hard)':[],'spark (hard)':[], 'hadoop (hard)':[], 'business intelligence (hard)':[],
                'machine learning (hard)': [], 'презентация  (soft)':[],'google analytics (hard)':[], 'oracle (hard)':[], 'powerpoint (hard)':[],
                'активность  (soft)':[], 'superset (hard)':[],'visualisation (hard)':[], 'работа в команде  (soft)':[],
                'git (hard)':[], 'критическое мышление  (soft)':[], 'яндекс.метрика (hard)':[], 'linux (hard)':[],
                'принятие решений  (soft)':[],'grafana (hard)':[], 'vba (hard)':[], 'api (hard)':[], 'внимательность (soft)':[],
                'greenplum (hard)':[], 'pyspark (hard)':[], 'elasticsearch (hard)':[], 'google docs (hard)':[],
                'data scince (hard)':[], 'hive (hard)':[], 'оценка рисков (hard)':[], 'креативность (soft)':[],
                'бизнес-показатели (hard)':[], 'product management (hard)':[], 'управление  (soft)':[], 'маркетинг':[], 'supply (hard)':[], 'other':[]}

In [ ]:
for skill, count in skill_sen_count.items():
    for _ in range(count):
        if 'sql' in skill:
            skill_sen_groups['sql (hard)'].append(skill)
        elif 'python' in skill or 'pandas' in skill or 'питон' in skill or 'scipy' in skill or 'plotly' in skill or 'seborn' in skill or 'matplotlib' in skill:
            skill_sen_groups['python (hard)'].append(skill)
        elif 'analy' in skill or 'анали' in skill or 'гипот' in skill or 'olap' in skill:
            skill_sen_groups['data analysis (hard)'].append(skill)
        elif 'документ' in skill or 'отчет' in skill:
            skill_sen_groups['документация (soft)'].append(skill)
        elif 'комму' in skill or 'речь' in skill or 'пис' in skill or 'грамот' in skill:
            skill_sen_groups['коммуникация (soft)'].append(skill)
        elif 'power' in skill and 'bi' in skill:
            skill_sen_groups['powerbi (hard)'].append(skill)
        elif 'tableau' in skill:
            skill_sen_groups['tableau (hard)'].append(skill)
        elif 'тест' in skill and 'a' in skill:
            skill_sen_groups['a/b тесты (hard)'].append(skill)
        elif 'статис' in skill or 'statis' in skill:
            skill_sen_groups['статистика (hard)'].append(skill)
        elif 'confl' in skill:
            skill_sen_groups['confluence (hard)'].append(skill)
        elif 'clickh' in skill:
            skill_sen_groups['clickhouse (hard)'].append(skill)
        elif 'юнит' in skill or 'unit' in skill:
            skill_sen_groups['юнит-экономика (hard)'].append(skill)
        elif 'exc' in skill:
            skill_sen_groups['excel (hard)'].append(skill)
        elif 'баз' in skill or 'субд' in skill or 'base' in skill:
            skill_sen_groups['базы данных (hard)'].append(skill)
        elif 'big data' in skill or 'больш' in skill:
            skill_sen_groups['работа с большим объемом информации (soft)'].append(skill)
        elif 'etl' in skill:
            skill_sen_groups['etl (hard)'].append(skill)
        elif 'англ' in skill or 'engl' in skill:
            skill_sen_groups['английский язык (hard)'].append(skill)
        elif 'airflow' in skill:
            skill_sen_groups['airflow (hard)'].append(skill)
        elif 'spark' in skill:
            skill_sen_groups['spark (hard)'].append(skill)
        elif 'hadoop' in skill:
            skill_sen_groups['hadoop (hard)'].append(skill)
        elif 'intelli' in skill or 'bi' in skill or 'sap' in skill:
            skill_sen_groups['business intelligence (hard)'].append(skill)
        elif 'learning' in skill or 'машин' in skill or skill=='ml':
            skill_sen_groups['machine learning (hard)'].append(skill)
        elif 'презент' in skill:
            skill_sen_groups['презентация  (soft)'].append(skill)
        elif 'goog' in skill and 'anal' in skill:
            skill_sen_groups['google analytics (hard)'].append(skill)
        elif 'oracle' in skill:
            skill_sen_groups['oracle (hard)'].append(skill)
        elif 'power' in skill and 'point' in skill:
            skill_sen_groups['powerpoint (hard)'].append(skill)
        elif 'активн' in skill:
            skill_sen_groups['активность  (soft)'].append(skill)
        elif 'superset' in skill:
            skill_sen_groups['superset (hard)'].append(skill)
        elif 'visua' in skill or 'визуа' in skill:
            skill_sen_groups['visualisation (hard)'].append(skill)
        elif 'команд' in skill:
            skill_sen_groups['работа в команде  (soft)'].append(skill)
        elif 'git' in skill:
            skill_sen_groups['git (hard)'].append(skill)
        elif 'крити' in skill:
            skill_sen_groups['критическое мышление  (soft)'].append(skill)
        elif 'метр' in skill and 'янд' in skill or 'yan' in skill:
            skill_sen_groups['яндекс.метрика (hard)'].append(skill)
        elif 'linux' in skill:
            skill_sen_groups['linux (hard)'].append(skill)
        elif 'реш' in skill:
            skill_sen_groups['принятие решений  (soft)'].append(skill)
        elif 'grafana' in skill:
            skill_sen_groups['grafana (hard)'].append(skill)
        elif 'vba' in skill:
            skill_sen_groups['vba (hard)'].append(skill)
        elif 'api' in skill:
            skill_sen_groups['api (hard)'].append(skill)
        elif 'внима' in skill:
            skill_sen_groups['внимательность (soft)'].append(skill)
        elif 'greenplum' in skill:
            skill_sen_groups['greenplum (hard)'].append(skill)
        elif 'pyspark' in skill:
            skill_sen_groups['pyspark (hard)'].append(skill)
        elif 'elasticsearch' in skill:
            skill_sen_groups['elasticsearch (hard)'].append(skill)
        elif 'google' in skill:
            skill_sen_groups['google docs (hard)'].append(skill)
        elif 'data' in skill and 'sci' in skill or 'nlp' in skill:
            skill_sen_groups['data scince (hard)'].append(skill)
        elif 'hive' in skill:
            skill_sen_groups['hive (hard)'].append(skill)
        elif 'риск' in skill or 'risk' in skill:
            skill_sen_groups['оценка рисков (hard)'].append(skill)
        elif 'креа' in skill:
            skill_sen_groups['креативность (soft)'].append(skill)
        elif 'показ' in skill or 'roi' in skill or 'cpl' in skill or 'cpa' in skill or 'cor' in skill or 'ltv' in skill:
            skill_sen_groups['бизнес-показатели (hard)'].append(skill)
        elif 'продукт' in skill or 'product' in skill:
            skill_sen_groups['product management (hard)'].append(skill)
        elif 'управл' in skill or 'лидер' in skill or 'руков' in skill or 'руков' in skill or 'организа' in skill or 'managem' in skill:
            skill_sen_groups['управление  (soft)'].append(skill)
        elif 'маркетинг' in skill or 'marketi' in skill or 'smm' in skill or 'rtb' in skill or 'реклам' in skill or 'лидо' in skill:
            skill_sen_groups['маркетинг'].append(skill)
        elif 'supply' in skill or 'запас' in skill or 'demand' in skill or 'складс' in skill:
            skill_sen_groups['supply (hard)'].append(skill)
        else:
            skill_sen_groups["other"].append(skill)


skills_ad_sen_df = pd.DataFrame({"skill": list(skill_sen_groups.keys()), "count_sen": [len(value) for value in skill_sen_groups.values()]})
skills_ad_sen_df = skills_ad_sen_df.sort_values(by='skill').reset_index(drop=True)
skills_ad_sen_df.head()

,skill,count_sen
0,a/b тесты (hard),4
1,airflow (hard),1
2,api (hard),4
3,business intelligence (hard),1
4,clickhouse (hard),1


#### Объединённые данные по скиллам DA

In [ ]:
#объединим получившиеся данные и разделим на два датафрейма: хард и софт скиллы
skills_final_da = pd.merge(skills_ad_df, skills_ad_jr_df, on='skill')
skills_final_da = pd.merge(skills_final_da, skills_ad_jrp_df, on='skill')
skills_final_da = pd.merge(skills_final_da, skills_ad_mid_df, on='skill')
skills_final_da = pd.merge(skills_final_da, skills_ad_sen_df, on='skill').sort_values(by='count', ascending=False).reset_index(drop=True)

skills_final_da.columns = ['skill', 'total', 'jr', 'jr+', 'mid', 'sen']
skills_final_da

,skill,total,jr,jr+,mid,sen
0,data analysis (hard),741,33,390,299,19
1,python (hard),608,20,272,299,17
2,sql (hard),550,13,264,257,16
3,other,481,18,214,231,18
4,работа с большим объемом информации (soft),127,1,65,60,1
5,документация (soft),126,3,62,55,6
6,коммуникация (soft),120,4,56,55,5
7,powerbi (hard),116,4,72,39,1
8,базы данных (hard),103,4,65,31,3
9,tableau (hard),74,0,41,33,0


In [ ]:
hard_skills_final_da = skills_final_da[skills_final_da['skill'].str.contains('hard')].reset_index(drop=True)
hard_skills_final_da

,skill,total,jr,jr+,mid,sen
0,data analysis (hard),741,33,390,299,19
1,python (hard),608,20,272,299,17
2,sql (hard),550,13,264,257,16
3,powerbi (hard),116,4,72,39,1
4,базы данных (hard),103,4,65,31,3
5,tableau (hard),74,0,41,33,0
6,статистика (hard),67,5,46,13,3
7,a/b тесты (hard),66,0,27,35,4
8,confluence (hard),61,2,32,25,2
9,excel (hard),51,3,36,10,2


In [ ]:
hard_skills_final_da.to_csv('hard_skills_final_da.csv', index = False)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
soft_skills_final_da = skills_final_da[skills_final_da['skill'].str.contains('soft')].reset_index(drop=True)
soft_skills_final_da

,skill,total,jr,jr+,mid,sen
0,работа с большим объемом информации (soft),127,1,65,60,1
1,документация (soft),126,3,62,55,6
2,коммуникация (soft),120,4,56,55,5
3,принятие решений (soft),26,0,7,16,3
4,презентация (soft),25,2,9,9,5
5,управление (soft),18,1,7,4,6
6,работа в команде (soft),17,0,9,8,0
7,активность (soft),16,1,9,5,1
8,критическое мышление (soft),11,1,0,7,3
9,внимательность (soft),8,1,3,2,2


In [ ]:
soft_skills_final_da.to_csv('soft_skills_final_da.csv', index = False)

### Требуемый опыт

In [ ]:
#сгруппируем по грейдам имеющиеся данные об опыте
exp_req = da[['grade','experience_year']]
exp_req['count']=''

exp_req = exp_req.pivot_table(index= 'grade', columns='experience_year', values='experience_year', aggfunc='count').reset_index()
exp_req = exp_req.rename_axis(None, axis=1)
exp_req.transpose()

<ipython-input-37-01c8c7ef6f70>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exp_req['count']=''


,0,1,2,3
grade,junior,junior+,middle,senior
1-3,4.0,378.0,5.0,NaN
3-6,2.0,NaN,347.0,6.0
6+,NaN,NaN,NaN,19.0
no experince,25.0,NaN,NaN,NaN


In [ ]:
exp_req.to_csv('exp_req.csv', index = False)

### Определение типичного места работы DA

Определение типичного места работы для АД и БА по следующим параметрам: ТОП-работодателей, зарплата, тип занятости, график работы.

#### Топ работодателей

In [ ]:
employer_da_total = da.groupby('employer')['id'].count().sort_values(ascending=False)
employer_da_jr = da[da['grade'] == 'junior'].groupby('employer')['id'].count()
employer_da_jrp = da[da['grade'] == 'junior+'].groupby('employer')['id'].count()
employer_da_mid = da[da['grade'] == 'middle'].groupby('employer')['id'].count()
employer_da_sen = da[da['grade'] == 'senior'].groupby('employer')['id'].count()

employer_da = pd.concat([employer_da_total, employer_da_jr, employer_da_jrp, employer_da_mid, employer_da_sen], axis=1).reset_index()
employer_da.columns = ['employer','total', 'jr', 'jr+', 'mid', 'sen']
employer_da.head(30)

,employer,total,jr,jr+,mid,sen
0,СБЕР,101,4.0,65.0,32.0,NaN
1,WILDBERRIES,36,NaN,13.0,23.0,NaN
2,МТС,32,1.0,25.0,6.0,NaN
3,Playrix,21,NaN,NaN,21.0,NaN
4,КАЗАНЬЭКСПРЕСС,10,NaN,3.0,7.0,NaN
5,«UZUM TECHNOLOGIES».,10,NaN,NaN,10.0,NaN
6,"МАГНИТ, Розничная сеть",10,NaN,10.0,NaN,NaN
7,Райффайзен Банк,9,NaN,2.0,6.0,1.0
8,USETECH,9,NaN,NaN,7.0,2.0
9,Maxim technology,8,NaN,NaN,8.0,NaN


In [ ]:
employer_da.to_csv('employer_da.csv', index = False)

#### Зарплата

In [ ]:
#изучим данные по ЗП у джунов без опыта
salary_jr_da = da[da['grade'] == 'junior'].describe()
salary_jr_da = salary_jr_da.drop(['id'], axis=1)
salary_jr_da = salary_jr_da.drop(['std', '25%', '75%'], axis=0).reset_index()
salary_jr_da_raw = da[da['grade'] == 'junior']
salary_jr_da_raw = salary_jr_da_raw[['id','salary_from', 'salary_to']].reset_index(drop=True)
salary_jr_da

,index,salary_from,salary_to
0,count,6.000000,9.000000
1,mean,78333.333333,122372.222222
2,min,40000.000000,40000.000000
3,50%,65000.000000,80000.000000
4,max,150000.000000,350000.000000


In [ ]:
salary_jr_da_raw.to_csv('salary_jr_da_raw.csv', index = False)

In [ ]:
#изучим данные по ЗП у джунов с опытом 1-3 года
salary_jrp_da = da[da['grade'] == 'junior+'].describe()
salary_jrp_da = salary_jrp_da.drop(['id'], axis=1)
salary_jrp_da = salary_jrp_da.drop(['std', '25%', '75%'], axis=0).reset_index()
salary_jrp_da_raw = da[da['grade'] == 'junior+']
salary_jrp_da_raw = salary_jrp_da_raw[['id','salary_from', 'salary_to']].reset_index(drop=True)
salary_jrp_da

,index,salary_from,salary_to
0,count,27.000000,36.000000
1,mean,139305.555556,181986.111111
2,min,20000.000000,46750.000000
3,50%,120000.000000,190000.000000
4,max,280500.000000,374000.000000


In [ ]:
salary_jrp_da_raw.to_csv('salary_jrp_da_raw.csv', index = False)

In [ ]:
#изучим данные по ЗП у миддлов
salary_mid_da = da[da['grade'] == 'middle'].describe()
salary_mid_da = salary_mid_da.drop(['id'], axis=1)
salary_mid_da = salary_mid_da.drop(['std', '25%', '75%'], axis=0).reset_index()
salary_mid_da_raw = da[da['grade'] == 'middle']
salary_mid_da_raw = salary_mid_da_raw[['id','salary_from', 'salary_to']].reset_index(drop=True)
salary_mid_da

,index,salary_from,salary_to
0,count,28.000000,35.000000
1,mean,190533.928571,295658.571429
2,min,84150.000000,121550.000000
3,50%,180000.000000,280500.000000
4,max,398000.000000,561000.000000


In [ ]:
salary_mid_da_raw.to_csv('salary_mid_da_raw.csv', index = False)

In [ ]:
#изучим данные по ЗП у синьйоров
salary_sen_da = da[da['grade'] == 'senior'].describe()
salary_sen_da = salary_sen_da.drop(['id'], axis=1)
salary_sen_da = salary_sen_da.drop(['std', '25%', '75%'], axis=0).reset_index()
salary_sen_da_raw = da[da['grade'] == 'senior']
salary_sen_da_raw = salary_sen_da_raw[['id','salary_from', 'salary_to']].reset_index(drop=True)
salary_sen_da

,index,salary_from,salary_to
0,count,8.00,6.000000
1,mean,247718.75,476166.666667
2,min,200000.00,374000.000000
3,50%,233750.00,480500.000000
4,max,300000.00,561000.000000


In [ ]:
salary_sen_da_raw.to_csv('salary_sen_da_raw.csv', index = False)

#### Тип занятости

In [ ]:
employment_jr = da[da['grade'] == 'junior'].groupby('employment')['id'].count()
employment_jrp = da[da['grade'] == 'junior+'].groupby('employment')['id'].count()
employment_mid = da[da['grade'] == 'middle'].groupby('employment')['id'].count()
employment_sen = da[da['grade'] == 'senior'].groupby('employment')['id'].count()
employment_da = pd.concat([employment_jr, employment_jrp, employment_mid, employment_sen], axis=1).reset_index()
employment_da = employment_da.fillna(0)
employment_da.columns = ['Тип занятости', 'jr', 'jr+', 'mid', 'sen']
employment_da

,Тип занятости,jr,jr+,mid,sen
0,Полная занятость,29.0,375.0,349.0,24.0
1,Стажировка,2.0,0.0,0.0,0.0
2,Проектная работа,0.0,1.0,0.0,1.0
3,Частичная занятость,0.0,2.0,3.0,0.0


In [ ]:
employment_da.to_csv('employment_da.csv', index = False)

#### График работы

In [ ]:
schedule_jr = da[da['grade'] == 'junior'].groupby('schedule')['id'].count()
schedule_jrp = da[da['grade'] == 'junior+'].groupby('schedule')['id'].count()
schedule_mid = da[da['grade'] == 'middle'].groupby('schedule')['id'].count()
schedule_sen = da[da['grade'] == 'senior'].groupby('schedule')['id'].count()
schedule_da = pd.concat([schedule_jr, schedule_jrp, schedule_mid, schedule_sen], axis=1).reset_index()
schedule_da = schedule_da.fillna(0)
schedule_da.columns = ['Расписание', 'jr', 'jr+', 'mid', 'sen']
schedule_da

,Расписание,jr,jr+,mid,sen
0,Гибкий график,1.0,14,10.0,1.0
1,Полный день,26.0,302,257.0,13.0
2,Удаленная работа,4.0,61,85.0,11.0
3,Сменный график,0.0,1,0.0,0.0


In [ ]:
schedule_da.to_csv('schedule_da.csv', index = False)

### Помесячная динамика количества вакансий для DA

Расчёт помесячной динамики количества вакансий.

In [ ]:
vac_jr = da[da['experience'] == 'Junior (no experince)'].groupby('month')['id'].count()
vac_jrp = da[da['experience'] == 'Junior+ (1-3 years)'].groupby('month')['id'].count()
vac_mid = da[da['experience'] == 'Middle (3-6 years)'].groupby('month')['id'].count()
vac_sen = da[da['experience'] == 'Senior (6+ years)'].groupby('month')['id'].count()
vac_da = pd.concat([vac_jr, vac_jrp, vac_mid, vac_sen], axis=1).reset_index()
vac_da.columns = ['Месяц', 'jr', 'jr+', 'mid', 'sen']
vac_da

,Месяц,jr,jr+,mid,sen
0,01/2024,7,122,95,6.0
1,09/2023,1,10,4,NaN
2,10/2023,6,97,83,5.0
3,11/2023,6,84,89,3.0
4,12/2023,5,74,84,5.0


In [ ]:
vac_da.to_csv('vac_da.csv', index = False)

## Предобработка BA

In [ ]:
ba_raw.head()

,id,name,published_at,alternate_url,type,employer,department,area,experience,key_skills,schedule,employment,description,description_lemmatized,salary_from,salary_to,salary_bin,key_skills_from_key_skills_field,hard_skills_from_description,soft_skills_from_description
0,49470036,Бизнес-аналитик,2024-01-29 21:09:56,https://hh.ru/vacancy/49470036,Открытая,ICS Consulting,NaN,Москва,Junior+ (1-3 years),"['Умение работать с людьми', 'Документация', '...",Полный день,Полная занятость,на основе конкурсного отбора обязанности учас...,основа конкурсный отбор обязанность участие ко...,90000.0,120000.0,От 100 тысяч до 200 тысяч,"Грамотность, Деловая переписка, Умение работат...",NaN,"Коммуникация, Документация, Переговоры"
1,55186681,"Проектный бизнес-аналитик 1С (ERP, УХ, КА, УТ,...",2024-01-10 11:00:30,https://hh.ru/vacancy/55186681,Открытая,Первый Бит,NaN,Москва,Middle (3-6 years),"['1С: Документооборот', '1С: Управление Произв...",Полный день,Полная занятость,"привет, будущий коллега!мы приглашаем проектны...",привет будущий коллега приглашать проектный ан...,NaN,NaN,ЗП не указана,"Управление проектами, Бизнес-анализ, Системный...",NaN,NaN
2,67043806,Бизнес-аналитик,2023-11-15 17:46:22,https://hh.ru/vacancy/67043806,Открытая,Digital Design,NaN,Москва,Junior+ (1-3 years),"['Разработка проектной документации', 'Электро...",Полный день,Полная занятость,digital design более четверти века разрабатыва...,digital design четверть век разрабатывать внед...,NaN,NaN,ЗП не указана,"Разработка проектной документации, Разработка ...",NaN,"Коммуникация, Документация"
3,67345392,"Бизнес-аналитик (по логистике морских портов, ...",2024-01-17 12:53:15,https://hh.ru/vacancy/67345392,Открытая,СОЛВО,NaN,Санкт-Петербург,Middle (3-6 years),"['IDEF0', 'Документация', 'Аналитическое мышле...",Полный день,Полная занятость,солво – лидер российского рынка it решений для...,солво лидер российский рынок it решение автома...,NaN,NaN,ЗП не указана,"UML, BPMN, IDEF0, Английский язык, IDEF",NaN,"Документация, Аналитическое мышление"
4,67498251,Бизнес-аналитик в Банк,2024-01-29 12:05:14,https://hh.ru/vacancy/67498251,Открытая,Сателл ИТ,NaN,Москва,Junior+ (1-3 years),"['Бизнес-анализ', 'Документация']",Удаленная работа,Полная занятость,"обязанности: анализ бизнес-процессов; сбор, а...",обязанность анализ бизнес процесс сбор анализ ...,200000.0,NaN,ЗП не указана,Бизнес-анализ,NaN,Документация


In [ ]:
#уберем некоторые столбцы, которые нам не пригодятся при анализе

ba = ba_raw.drop(['alternate_url', 'type', 'department'], axis=1)
ba.head()

,id,name,published_at,employer,area,experience,key_skills,schedule,employment,description,description_lemmatized,salary_from,salary_to,salary_bin,key_skills_from_key_skills_field,hard_skills_from_description,soft_skills_from_description
0,49470036,Бизнес-аналитик,2024-01-29 21:09:56,ICS Consulting,Москва,Junior+ (1-3 years),"['Умение работать с людьми', 'Документация', '...",Полный день,Полная занятость,на основе конкурсного отбора обязанности учас...,основа конкурсный отбор обязанность участие ко...,90000.0,120000.0,От 100 тысяч до 200 тысяч,"Грамотность, Деловая переписка, Умение работат...",NaN,"Коммуникация, Документация, Переговоры"
1,55186681,"Проектный бизнес-аналитик 1С (ERP, УХ, КА, УТ,...",2024-01-10 11:00:30,Первый Бит,Москва,Middle (3-6 years),"['1С: Документооборот', '1С: Управление Произв...",Полный день,Полная занятость,"привет, будущий коллега!мы приглашаем проектны...",привет будущий коллега приглашать проектный ан...,NaN,NaN,ЗП не указана,"Управление проектами, Бизнес-анализ, Системный...",NaN,NaN
2,67043806,Бизнес-аналитик,2023-11-15 17:46:22,Digital Design,Москва,Junior+ (1-3 years),"['Разработка проектной документации', 'Электро...",Полный день,Полная занятость,digital design более четверти века разрабатыва...,digital design четверть век разрабатывать внед...,NaN,NaN,ЗП не указана,"Разработка проектной документации, Разработка ...",NaN,"Коммуникация, Документация"
3,67345392,"Бизнес-аналитик (по логистике морских портов, ...",2024-01-17 12:53:15,СОЛВО,Санкт-Петербург,Middle (3-6 years),"['IDEF0', 'Документация', 'Аналитическое мышле...",Полный день,Полная занятость,солво – лидер российского рынка it решений для...,солво лидер российский рынок it решение автома...,NaN,NaN,ЗП не указана,"UML, BPMN, IDEF0, Английский язык, IDEF",NaN,"Документация, Аналитическое мышление"
4,67498251,Бизнес-аналитик в Банк,2024-01-29 12:05:14,Сателл ИТ,Москва,Junior+ (1-3 years),"['Бизнес-анализ', 'Документация']",Удаленная работа,Полная занятость,"обязанности: анализ бизнес-процессов; сбор, а...",обязанность анализ бизнес процесс сбор анализ ...,200000.0,NaN,ЗП не указана,Бизнес-анализ,NaN,Документация


In [ ]:
ba.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4171 entries, 0 to 4170
Data columns (total 17 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   id                                4171 non-null   int64  
 1   name                              4171 non-null   object 
 2   published_at                      4171 non-null   object 
 3   employer                          4171 non-null   object 
 4   area                              4171 non-null   object 
 5   experience                        4171 non-null   object 
 6   key_skills                        4171 non-null   object 
 7   schedule                          4171 non-null   object 
 8   employment                        4171 non-null   object 
 9   description                       4171 non-null   object 
 10  description_lemmatized            4171 non-null   object 
 11  salary_from                       927 non-null    float64
 12  salary

Проведём предобработку данных во фрейме ba

Изучим данные в числовых столбцах

In [ ]:
ba.describe()

,id,salary_from,salary_to
count,4.171000e+03,927.000000,633.000000
mean,8.969242e+07,109891.568501,152817.911532
std,2.558834e+06,62102.026151,78582.253331
min,4.947004e+07,60.000000,30000.000000
25%,8.888015e+07,69825.000000,90000.000000
50%,9.003147e+07,95000.000000,140000.000000
75%,9.132282e+07,150000.000000,200000.000000
max,9.241942e+07,350000.000000,467500.000000


Проверим дубликаты

In [ ]:
ba.duplicated().sum()
#явных дубликатов нет

0

In [ ]:
print(ba['area'].unique())
print(ba['experience'].unique())
print(ba['schedule'].unique())
print(ba['employment'].unique())

['Москва' 'Санкт-Петербург' 'Чебоксары' 'Керчь' 'Минск' 'Владивосток'
 'Набережные Челны' 'Новосибирск' 'Воронеж' 'Астрахань' 'Ташкент'
 'Краснодар' 'Екатеринбург' 'Нижний Новгород' 'Алматы'
 'Троицк (Московская область)' 'Подольск (Московская область)' 'Самара'
 'Ижевск' 'Омск' 'Мыски' 'Казань' 'Рязань' 'Саратов' 'Старый Оскол'
 'Владимир' 'Курск' 'Пермь' 'Челябинск' 'Петрозаводск' 'Ногинск'
 'Новокузнецк' 'Большой Камень' 'Кострома' 'Ростов-на-Дону'
 'Иваново (Ивановская область)' 'Тверь' 'Домодедово' 'Ульяновск' 'Тюмень'
 'Балашиха' 'Красноярск' 'Волгоград' 'Барнаул' 'Астана' 'Череповец'
 'Тольятти' 'Мытищи' 'Липецк' 'Калининград' 'Уфа' 'Тула' 'Белгород'
 'Ярославль' 'Березовский (Свердловская область)' 'Томск' 'Иркутск'
 'Кемерово' 'Гродно' 'Пенза' 'Магнитогорск' 'Тбилиси' 'Таганрог'
 'Павловский Посад' 'Электросталь' 'Бишкек' 'Якутск' 'Гомель' 'Брест'
 'Киров (Кировская область)' 'Смоленск' 'Щелково' 'Южно-Сахалинск'
 'Великий Новгород' 'Зеленоград' 'Хабаровск' 'Кипр' 'Сербия'
 'Н

Преобразуем форматы данных

In [ ]:
ba['published_at'] = pd.to_datetime(ba['published_at']).dt.date

Cоздадим колонку с месяцем опубликования вакансии

In [ ]:
ba['month'] = pd.DatetimeIndex(ba['published_at'])
ba['month'] = ba['month'].dt.strftime('%m/%Y')

In [ ]:
#создадим поле с грейдом из названия вакансии
ba['grade'] = ''

for i, title in enumerate(ba['name'].tolist()):
    if 'junior' in str(title):
        ba.loc[i, 'grade'] = 'junior'
    elif 'middle' in str(title):
        ba.loc[i, 'grade'] = 'middle'
    elif 'senior' in str(title):
        ba.loc[i, 'grade'] = 'senior'
    else:
      ba.loc[i, 'grade'] = np.NaN

In [ ]:
#не везде совпадают колонка опыт и грейд. Пустые ячейки в grade заполним значениями из experience
ba['grade'] = ba['grade'].T.fillna(ba['experience']).T
ba['grade'] = ba['grade'].apply(lambda row: row.split(' ')[0])
ba['grade'] = ba['grade'].apply(lambda x: x.lower())

In [ ]:
#создадим поле только с требованием опыта в годах
ba['experience_year'] = ba['experience'].apply(lambda row: row.split('years')[0])
ba['experience_year'] = ba['experience_year'].apply(lambda row: row.split('(')[1])
ba['experience_year'] = ba['experience_year'].str.replace(')','')

<ipython-input-65-d1bf00da5064>:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  ba['experience_year'] = ba['experience_year'].str.replace(')','')


В результате предобработки данных датасета по вакансиям Бизнес-аналитик были выполнены следубщие задачи:
1. Удалены данные, которые не пригодятся для исследования.
2. Изучены числовые показатели. Найдена одна аномалия в поле salary_from, которая будет исправлена при дальнейшем анализе данных.
3. Проведена проверка на явные и неявные дубликаты.
4. Преобразован формат некоторых данных для дальнейшей работы с ними.
5. Добавлены столбцы: месяц публикации вакансии, грейд (сделан на основе названия вакансии или требований к опыту).

## Анализ данных BA

### Анализ скиллов BA

#### Скиллы BA

Обработаем данные по скиллам

In [ ]:
#сделаем в рабочем поле с ключевыми навыками нижний регистр, преобразуем в список и уберём ненужные элементы
ba['key_skills_process'] = ba['key_skills'].str.lower()
ba['key_skills_process'] = ba['key_skills_process'].str.split(', ')
ba['key_skills_process'] = [[element.replace("[", "") for element in sublist] for sublist in ba['key_skills_process']]
ba['key_skills_process'] = [[element.replace("'", "") for element in sublist] for sublist in ba['key_skills_process']]
ba['key_skills_process'] = [[element.replace("]", "") for element in sublist] for sublist in ba['key_skills_process']]

In [ ]:
#создадим пустой словарь, куда будут добавляться значения из поля 'key_skills_process' в зависимости от названия скилла
skill_ba_count = {}

for row in ba['key_skills_process']:
    for element in row:
        if element in skill_ba_count:
            skill_ba_count[element] += 1
        else:
            skill_ba_count[element] = 1

print(skill_ba_count)

{'умение работать с людьми': 2, 'документация': 1942, 'коммуникация': 662, 'знание инструментов моделирования бизнес-процессов': 1, 'ms office': 28, 'умение работать в условиях многозадачности': 12, 'грамотность': 21, 'переговоры': 179, 'деловая переписка': 45, '1с: документооборот': 79, '1с: управление производственным предприятием': 42, '1с: управление холдингом': 3, '1c: финансы': 4, 'работа в команде': 122, 'бизнес-анализ': 1476, 'erp-системы на базе 1с': 158, 'бизнес-моделирование': 111, 'erp': 68, 'автоматизация процессов': 172, 'kanban': 11, 'управление проектами': 154, '1с: управление предприятием': 33, 'разработка технических заданий': 497, 'системный анализ': 242, 'разработка проектной документации': 35, 'электронный документооборот': 19, 'docsvision': 2, 'сэд': 13, 'проведение презентаций': 56, 'pre-sales': 1, 'idef0': 85, 'аналитическое мышление': 1027, 'английский язык': 134, 'uml': 391, 'bpmn': 771, 'idef': 60, 'ms sql': 109, 'системное мышление': 109, 'erp systems': 30, 

In [ ]:
#посмотрим в сыром виде, какие скиллы наиболее часто встречаются
skills_ba_raw = pd.DataFrame.from_dict(skill_ba_count, orient='index', columns=['count']).sort_values(by='count', ascending=False)

skills_ba_raw

,count
документация,1942
бизнес-анализ,1476
аналитическое мышление,1027
bpmn,771
коммуникация,662
...,...
функциональные и нефункциональные требования,1
авторская приемка,1
подготовка и реализация бизнес проектов,1
опыт общения с заказчиками,1


In [ ]:
#укрупним группы. За основу возьмём наиболее часто встречающиеся элементы
skill_ba_groups = {'бизнес-анализ (hard)':[], 'аналитическое мышление (soft)': [], 'bpmn (hard)':[],
                'знание бизнес процессов (hard)':[], 'разработка ТЗ, инструкций, регламентов (hard)':[],
                'uml (hard)':[], 'sql (hard)':[],'документация (soft)': [],
                'коммуникация (soft)':[], 'data analysis (hard)':[],'бизнес-показатели (hard)':[], 'excel (hard)':[],
                'jira (hard)':[], 'visio (hard)':[], 'автоматизация процессов (hard)':[], 'confluence (hard)':[], 'powerpoint (hard)':[],
                'erp-системы (hard)':[], 'управление  (soft)':[], 'английский язык (hard)':[], 'работа в команде (soft)':[],
                'powerbi (hard)':[], 'бизнес-моделирование (hard)':[], 'agile (hard)':[], 'системное мышление (soft)':[], '1с (hard)':[], 'scrum (hard)':[],
                'активность  (soft)':[], 'crm (hard)':[], 'idef0 (hard)':[], 'python (hard)':[], 'базы данных (hard)':[], 'epc (hard)':[],
                'бухгалтерский учет (hard)':[], 'idef (hard)':[], 'презентация  (soft)':[], 'прогнозирование (hard)':[],
                'юнит-экономика (hard)':[], 'api (hard)':[],'креативность (soft)':[], 'cjm (hard)':[], 'user story (hard)':[],
                'обучение и развитие (soft)':[], 'tableau (hard)':[], 'бизнес-консультирование (hard)':[], 'оценка рисков (hard)':[],
                'работа с большим объемом информации (soft)':[], 'product management (hard)':[], 'принятие решений  (soft)':[],
                'статистика (hard)':[],'работа в команде  (soft)':[], 'критическое мышление  (soft)':[],
                'sap (hard)':[], 'other':[]}

In [ ]:
#по ключевым словам объединим скиллы по группам
for skill, count in skill_ba_count.items():
    for _ in range(count):
        if 'sql' in skill:
            skill_ba_groups['sql (hard)'].append(skill)
        elif 'python' in skill or 'pandas' in skill or 'питон' in skill or 'scipy' in skill or 'plotly' in skill or 'seborn' in skill or 'matplotlib' in skill:
            skill_ba_groups['python (hard)'].append(skill)
        elif ('analy' in skill or 'анали' in skill or 'гипот' in skill or 'olap' in skill) and 'business' not in skill and 'бизнес' not in skill and 'financial' not in skill and '1с аналитик' not in skill and 'мышл' not in skill:
            skill_ba_groups['data analysis (hard)'].append(skill)
        elif ('документ' in skill or 'отчет' in skill or 'договор' in skill) and '1с: документооборот' not in skill and 'разраб' not in skill:
            skill_ba_groups['документация (soft)'].append(skill)
        elif ('комму' in skill or 'речь' in skill or 'пис' in skill or 'грамот' in skill) and 'задан' not in skill and 'напис' not in skill:
            skill_ba_groups['коммуникация (soft)'].append(skill)
        elif 'power' in skill and 'bi' in skill:
            skill_ba_groups['powerbi (hard)'].append(skill)
        elif 'tableau' in skill:
            skill_ba_groups['tableau (hard)'].append(skill)
        elif 'статис' in skill or 'statis' in skill:
            skill_ba_groups['статистика (hard)'].append(skill)
        elif 'confl' in skill:
            skill_ba_groups['confluence (hard)'].append(skill)
        elif 'юнит' in skill or 'unit' in skill:
            skill_ba_groups['юнит-экономика (hard)'].append(skill)
        elif 'exc' in skill:
            skill_ba_groups['excel (hard)'].append(skill)
        elif 'баз' in skill or 'субд' in skill or 'base' in skill:
            skill_ba_groups['базы данных (hard)'].append(skill)
        elif 'англ' in skill or 'engl' in skill:
            skill_ba_groups['английский язык (hard)'].append(skill)
        elif 'презент' in skill:
            skill_ba_groups['презентация  (soft)'].append(skill)
        elif 'power' in skill and 'point' in skill:
            skill_ba_groups['powerpoint (hard)'].append(skill)
        elif ('активн' in skill) and ('клиент' not in skill):
            skill_ba_groups['активность  (soft)'].append(skill)
        elif ('команд' in skill) and ('управл' not in skill) and ('руковод' not in skill):
            skill_ba_groups['работа в команде  (soft)'].append(skill)
        elif 'крити' in skill:
            skill_ba_groups['критическое мышление  (soft)'].append(skill)
        elif 'реш' in skill:
            skill_ba_groups['принятие решений  (soft)'].append(skill)
        elif 'api' in skill:
            skill_ba_groups['api (hard)'].append(skill)
        elif 'риск' in skill or 'risk' in skill:
            skill_ba_groups['оценка рисков (hard)'].append(skill)
        elif 'креа' in skill:
            skill_ba_groups['креативность (soft)'].append(skill)
        elif 'показ' in skill or 'roi' in skill or 'cpl' in skill or 'cpa' in skill or 'cor' in skill or 'ltv' in skill:
            skill_ba_groups['бизнес-показатели (hard)'].append(skill)
        elif ('продукт' in skill or 'product' in skill) and '1с' not in skill:
            skill_ba_groups['product management (hard)'].append(skill)
        elif ('управл' in skill or 'лидер' in skill or 'руков' in skill or 'руков' in skill or 'организа' in skill or 'managem' in skill) and '1c' not in skill and '1с' not in skill and 'консульт' not in skill and 'поручен' not in skill and 'консульт' not in skill and 'внедрен' not in skill and '!с' not in skill and 'разраб' not in skill:
            skill_ba_groups['управление  (soft)'].append(skill)
        elif ('бизнес' in skill and 'анали' in skill) or ('busin' in skill and 'anal' in skill) or ('finan' in skill and 'anal' in skill):
            skill_ba_groups['бизнес-анализ (hard)'].append(skill)
        elif 'анали' in skill and 'мышл' in skill:
            skill_ba_groups['аналитическое мышление (soft)'].append(skill)
        elif 'bpmn' in skill:
            skill_ba_groups['bpmn (hard)'].append(skill)
        elif ('бизнес' in skill and 'процес' in skill) or ('busin' in skill and 'proc' in skill):
            skill_ba_groups['знание бизнес процессов (hard)'].append(skill)
        elif ('разраб' in skill and 'задан' in skill or 'тз' in skill or 'инструкц' in skill or 'реглам' in skill) and 'процес' not in skill and 'учет' not in skill:
            skill_ba_groups['разработка ТЗ, инструкций, регламентов (hard)'].append(skill)
        elif 'uml' in skill:
            skill_ba_groups['uml (hard)'].append(skill)
        elif 'jira' in skill:
            skill_ba_groups['jira (hard)'].append(skill)
        elif 'visio' in skill:
            skill_ba_groups['visio (hard)'].append(skill)
        elif 'автоматиз' in skill and 'процес' in skill:
            skill_ba_groups['автоматизация процессов (hard)'].append(skill)
        elif 'erp' in skill:
            skill_ba_groups['erp-системы (hard)'].append(skill)
        elif 'модел' in skill:
            skill_ba_groups['бизнес-моделирование (hard)'].append(skill)
        elif 'agile' in skill:
            skill_ba_groups['agile (hard)'].append(skill)
        elif 'сист' in skill and 'мышл' in skill:
            skill_ba_groups['системное мышление (soft)'].append(skill)
        elif '1c' in skill or '1с' in skill or '!c' in skill or '!с' in skill:
            skill_ba_groups['1с (hard)'].append(skill)
        elif 'scrum' in skill:
            skill_ba_groups['scrum (hard)'].append(skill)
        elif 'crm' in skill:
            skill_ba_groups['crm (hard)'].append(skill)
        elif 'idef' in skill:
            skill_ba_groups['idef0 (hard)'].append(skill)
        elif 'epc' in skill:
            skill_ba_groups['epc (hard)'].append(skill)
        elif 'бухг' in skill:
            skill_ba_groups['бухгалтерский учет (hard)'].append(skill)
        elif 'idef' in skill:
            skill_ba_groups['idef (hard)'].append(skill)
        elif 'прогноз' in skill:
            skill_ba_groups['прогнозирование (hard)'].append(skill)
        elif 'cjm' in skill:
            skill_ba_groups['cjm (hard)'].append(skill)
        elif 'user story' in skill:
            skill_ba_groups['user story (hard)'].append(skill)
        elif 'обуч' in skill or 'развит' in skill:
            skill_ba_groups['обучение и развитие (soft)'].append(skill)
        elif 'консульт' in skill or 'consult' in skill:
            skill_ba_groups['бизнес-консультирование (hard)'].append(skill)
        elif 'больш' in skill and 'инф' in skill:
            skill_ba_groups['работа с большим объемом информации (soft)'].append(skill)
        elif 'прин' in skill and 'реш' in skill:
            skill_ba_groups['принятие решений  (soft)'].append(skill)
        elif 'крит' in skill and 'мыш' in skill:
            skill_ba_groups['критическое мышление (soft)'].append(skill)
        elif 'sap' in skill:
            skill_ba_groups['sap (hard)'].append(skill)
        else:
            skill_ba_groups["other"].append(skill)


skills_ba_df = pd.DataFrame({"skill": list(skill_ba_groups.keys()), "count": [len(value) for value in skill_ba_groups.values()]})
skills_ba_df = skills_ba_df.sort_values(by='skill').reset_index(drop=True)

#### Скиллы с разбивкой на грейды

Проделаем то же самое отдельно по каждому грейду.

##### Junior

In [ ]:
skill_ba_jr_count = {}

for index, row in ba.iterrows():
    if row['grade'] == 'junior':
        for skill in row['key_skills_process']:
            if skill in skill_ba_jr_count:
                skill_ba_jr_count[skill] += 1
            else:
                skill_ba_jr_count[skill] = 1

In [ ]:
#укрупним группы. За основу возьмём наиболее часто встречающиеся элементы
skill_ba_jr_groups = {'бизнес-анализ (hard)':[], 'аналитическое мышление (soft)': [], 'bpmn (hard)':[],
                'знание бизнес процессов (hard)':[], 'разработка ТЗ, инструкций, регламентов (hard)':[],
                'uml (hard)':[], 'sql (hard)':[],'документация (soft)': [],
                'коммуникация (soft)':[], 'data analysis (hard)':[],'бизнес-показатели (hard)':[], 'excel (hard)':[],
                'jira (hard)':[], 'visio (hard)':[], 'автоматизация процессов (hard)':[], 'confluence (hard)':[], 'powerpoint (hard)':[],
                'erp-системы (hard)':[], 'управление  (soft)':[], 'английский язык (hard)':[], 'работа в команде (soft)':[],
                'powerbi (hard)':[], 'бизнес-моделирование (hard)':[], 'agile (hard)':[], 'системное мышление (soft)':[], '1с (hard)':[], 'scrum (hard)':[],
                'активность  (soft)':[], 'crm (hard)':[], 'idef0 (hard)':[], 'python (hard)':[], 'базы данных (hard)':[], 'epc (hard)':[],
                'бухгалтерский учет (hard)':[], 'idef (hard)':[], 'презентация  (soft)':[], 'прогнозирование (hard)':[],
                'юнит-экономика (hard)':[], 'api (hard)':[],'креативность (soft)':[], 'cjm (hard)':[], 'user story (hard)':[],
                'обучение и развитие (soft)':[], 'tableau (hard)':[], 'бизнес-консультирование (hard)':[], 'оценка рисков (hard)':[],
                'работа с большим объемом информации (soft)':[], 'product management (hard)':[], 'принятие решений  (soft)':[],
                'статистика (hard)':[],'работа в команде  (soft)':[], 'критическое мышление  (soft)':[],
                'sap (hard)':[], 'other':[]}

In [ ]:
for skill, count in skill_ba_jr_count.items():
    for _ in range(count):
        if 'sql' in skill:
            skill_ba_jr_groups['sql (hard)'].append(skill)
        elif 'python' in skill or 'pandas' in skill or 'питон' in skill or 'scipy' in skill or 'plotly' in skill or 'seborn' in skill or 'matplotlib' in skill:
            skill_ba_jr_groups['python (hard)'].append(skill)
        elif ('analy' in skill or 'анали' in skill or 'гипот' in skill or 'olap' in skill) and 'business' not in skill and 'бизнес' not in skill and 'financial' not in skill and '1с аналитик' not in skill and 'мышл' not in skill:
            skill_ba_jr_groups['data analysis (hard)'].append(skill)
        elif ('документ' in skill or 'отчет' in skill or 'договор' in skill) and '1с: документооборот' not in skill and 'разраб' not in skill:
            skill_ba_jr_groups['документация (soft)'].append(skill)
        elif ('комму' in skill or 'речь' in skill or 'пис' in skill or 'грамот' in skill) and 'задан' not in skill and 'напис' not in skill:
            skill_ba_jr_groups['коммуникация (soft)'].append(skill)
        elif 'power' in skill and 'bi' in skill:
            skill_ba_jr_groups['powerbi (hard)'].append(skill)
        elif 'tableau' in skill:
            skill_ba_jr_groups['tableau (hard)'].append(skill)
        elif 'статис' in skill or 'statis' in skill:
            skill_ba_jr_groups['статистика (hard)'].append(skill)
        elif 'confl' in skill:
            skill_ba_jr_groups['confluence (hard)'].append(skill)
        elif 'юнит' in skill or 'unit' in skill:
            skill_ba_jr_groups['юнит-экономика (hard)'].append(skill)
        elif 'exc' in skill:
            skill_ba_jr_groups['excel (hard)'].append(skill)
        elif 'баз' in skill or 'субд' in skill or 'base' in skill:
            skill_ba_jr_groups['базы данных (hard)'].append(skill)
        elif 'англ' in skill or 'engl' in skill:
            skill_ba_jr_groups['английский язык (hard)'].append(skill)
        elif 'презент' in skill:
            skill_ba_jr_groups['презентация  (soft)'].append(skill)
        elif 'power' in skill and 'point' in skill:
            skill_ba_jr_groups['powerpoint (hard)'].append(skill)
        elif ('активн' in skill) and ('клиент' not in skill):
            skill_ba_jr_groups['активность  (soft)'].append(skill)
        elif ('команд' in skill) and ('управл' not in skill) and ('руковод' not in skill):
            skill_ba_jr_groups['работа в команде  (soft)'].append(skill)
        elif 'крити' in skill:
            skill_ba_jr_groups['критическое мышление  (soft)'].append(skill)
        elif 'реш' in skill:
            skill_ba_jr_groups['принятие решений  (soft)'].append(skill)
        elif 'api' in skill:
            skill_ba_jr_groups['api (hard)'].append(skill)
        elif 'риск' in skill or 'risk' in skill:
            skill_ba_jr_groups['оценка рисков (hard)'].append(skill)
        elif 'креа' in skill:
            skill_ba_jr_groups['креативность (soft)'].append(skill)
        elif 'показ' in skill or 'roi' in skill or 'cpl' in skill or 'cpa' in skill or 'cor' in skill or 'ltv' in skill:
            skill_ba_jr_groups['бизнес-показатели (hard)'].append(skill)
        elif ('продукт' in skill or 'product' in skill) and '1с' not in skill:
            skill_ba_jr_groups['product management (hard)'].append(skill)
        elif ('управл' in skill or 'лидер' in skill or 'руков' in skill or 'руков' in skill or 'организа' in skill or 'managem' in skill) and '1c' not in skill and '1с' not in skill and 'консульт' not in skill and 'поручен' not in skill and 'консульт' not in skill and 'внедрен' not in skill and '!с' not in skill and 'разраб' not in skill:
            skill_ba_jr_groups['управление  (soft)'].append(skill)
        elif ('бизнес' in skill and 'анали' in skill) or ('busin' in skill and 'anal' in skill) or ('finan' in skill and 'anal' in skill):
            skill_ba_jr_groups['бизнес-анализ (hard)'].append(skill)
        elif 'анали' in skill and 'мышл' in skill:
            skill_ba_jr_groups['аналитическое мышление (soft)'].append(skill)
        elif 'bpmn' in skill:
            skill_ba_jr_groups['bpmn (hard)'].append(skill)
        elif ('бизнес' in skill and 'процес' in skill) or ('busin' in skill and 'proc' in skill):
            skill_ba_jr_groups['знание бизнес процессов (hard)'].append(skill)
        elif ('разраб' in skill and 'задан' in skill or 'тз' in skill or 'инструкц' in skill or 'реглам' in skill) and 'процес' not in skill and 'учет' not in skill:
            skill_ba_jr_groups['разработка ТЗ, инструкций, регламентов (hard)'].append(skill)
        elif 'uml' in skill:
            skill_ba_jr_groups['uml (hard)'].append(skill)
        elif 'jira' in skill:
            skill_ba_jr_groups['jira (hard)'].append(skill)
        elif 'visio' in skill:
            skill_ba_jr_groups['visio (hard)'].append(skill)
        elif 'автоматиз' in skill and 'процес' in skill:
            skill_ba_jr_groups['автоматизация процессов (hard)'].append(skill)
        elif 'erp' in skill:
            skill_ba_jr_groups['erp-системы (hard)'].append(skill)
        elif 'модел' in skill:
            skill_ba_jr_groups['бизнес-моделирование (hard)'].append(skill)
        elif 'agile' in skill:
            skill_ba_jr_groups['agile (hard)'].append(skill)
        elif 'сист' in skill and 'мышл' in skill:
            skill_ba_jr_groups['системное мышление (soft)'].append(skill)
        elif '1c' in skill or '1с' in skill or '!c' in skill or '!с' in skill:
            skill_ba_jr_groups['1с (hard)'].append(skill)
        elif 'scrum' in skill:
            skill_ba_jr_groups['scrum (hard)'].append(skill)
        elif 'crm' in skill:
            skill_ba_jr_groups['crm (hard)'].append(skill)
        elif 'idef' in skill:
            skill_ba_jr_groups['idef0 (hard)'].append(skill)
        elif 'epc' in skill:
            skill_ba_jr_groups['epc (hard)'].append(skill)
        elif 'бухг' in skill:
            skill_ba_jr_groups['бухгалтерский учет (hard)'].append(skill)
        elif 'idef' in skill:
            skill_ba_jr_groups['idef (hard)'].append(skill)
        elif 'прогноз' in skill:
            skill_ba_jr_groups['прогнозирование (hard)'].append(skill)
        elif 'cjm' in skill:
            skill_ba_jr_groups['cjm (hard)'].append(skill)
        elif 'user story' in skill:
            skill_ba_jr_groups['user story (hard)'].append(skill)
        elif 'обуч' in skill or 'развит' in skill:
            skill_ba_jr_groups['обучение и развитие (soft)'].append(skill)
        elif 'консульт' in skill or 'consult' in skill:
            skill_ba_jr_groups['бизнес-консультирование (hard)'].append(skill)
        elif 'больш' in skill and 'инф' in skill:
            skill_ba_jr_groups['работа с большим объемом информации (soft)'].append(skill)
        elif 'прин' in skill and 'реш' in skill:
            skill_ba_jr_groups['принятие решений  (soft)'].append(skill)
        elif 'крит' in skill and 'мыш' in skill:
            skill_ba_jr_groups['критическое мышление (soft)'].append(skill)
        elif 'sap' in skill:
            skill_ba_jr_groups['sap (hard)'].append(skill)
        else:
            skill_ba_jr_groups["other"].append(skill)


skills_ba_jr_df = pd.DataFrame({"skill": list(skill_ba_jr_groups.keys()), "count": [len(value) for value in skill_ba_jr_groups.values()]})
skills_ba_jr_df = skills_ba_jr_df.sort_values(by='skill').reset_index(drop=True)
skills_ba_jr_df.head()

,skill,count
0,1с (hard),174
1,agile (hard),1
2,api (hard),1
3,bpmn (hard),40
4,cjm (hard),0


##### Junior +

In [ ]:
skill_ba_jrp_count = {}

for index, row in ba.iterrows():
    if row['grade'] == 'junior+':
        for skill in row['key_skills_process']:
            if skill in skill_ba_jrp_count:
                skill_ba_jrp_count[skill] += 1
            else:
                skill_ba_jrp_count[skill] = 1

In [ ]:
#укрупним группы. За основу возьмём наиболее часто встречающиеся элементы
skill_ba_jrp_groups = {'бизнес-анализ (hard)':[], 'аналитическое мышление (soft)': [], 'bpmn (hard)':[],
                'знание бизнес процессов (hard)':[], 'разработка ТЗ, инструкций, регламентов (hard)':[],
                'uml (hard)':[], 'sql (hard)':[],'документация (soft)': [],
                'коммуникация (soft)':[], 'data analysis (hard)':[],'бизнес-показатели (hard)':[], 'excel (hard)':[],
                'jira (hard)':[], 'visio (hard)':[], 'автоматизация процессов (hard)':[], 'confluence (hard)':[], 'powerpoint (hard)':[],
                'erp-системы (hard)':[], 'управление  (soft)':[], 'английский язык (hard)':[], 'работа в команде (soft)':[],
                'powerbi (hard)':[], 'бизнес-моделирование (hard)':[], 'agile (hard)':[], 'системное мышление (soft)':[], '1с (hard)':[], 'scrum (hard)':[],
                'активность  (soft)':[], 'crm (hard)':[], 'idef0 (hard)':[], 'python (hard)':[], 'базы данных (hard)':[], 'epc (hard)':[],
                'бухгалтерский учет (hard)':[], 'idef (hard)':[], 'презентация  (soft)':[], 'прогнозирование (hard)':[],
                'юнит-экономика (hard)':[], 'api (hard)':[],'креативность (soft)':[], 'cjm (hard)':[], 'user story (hard)':[],
                'обучение и развитие (soft)':[], 'tableau (hard)':[], 'бизнес-консультирование (hard)':[], 'оценка рисков (hard)':[],
                'работа с большим объемом информации (soft)':[], 'product management (hard)':[], 'принятие решений  (soft)':[],
                'статистика (hard)':[],'работа в команде  (soft)':[], 'критическое мышление  (soft)':[],
                'sap (hard)':[], 'other':[]}

In [ ]:
for skill, count in skill_ba_jrp_count.items():
    for _ in range(count):
        if 'sql' in skill:
            skill_ba_jrp_groups['sql (hard)'].append(skill)
        elif 'python' in skill or 'pandas' in skill or 'питон' in skill or 'scipy' in skill or 'plotly' in skill or 'seborn' in skill or 'matplotlib' in skill:
            skill_ba_jrp_groups['python (hard)'].append(skill)
        elif ('analy' in skill or 'анали' in skill or 'гипот' in skill or 'olap' in skill) and 'business' not in skill and 'бизнес' not in skill and 'financial' not in skill and '1с аналитик' not in skill and 'мышл' not in skill:
            skill_ba_jrp_groups['data analysis (hard)'].append(skill)
        elif ('документ' in skill or 'отчет' in skill or 'договор' in skill) and '1с: документооборот' not in skill and 'разраб' not in skill:
            skill_ba_jrp_groups['документация (soft)'].append(skill)
        elif ('комму' in skill or 'речь' in skill or 'пис' in skill or 'грамот' in skill) and 'задан' not in skill and 'напис' not in skill:
            skill_ba_jrp_groups['коммуникация (soft)'].append(skill)
        elif 'power' in skill and 'bi' in skill:
            skill_ba_jrp_groups['powerbi (hard)'].append(skill)
        elif 'tableau' in skill:
            skill_ba_jrp_groups['tableau (hard)'].append(skill)
        elif 'статис' in skill or 'statis' in skill:
            skill_ba_jrp_groups['статистика (hard)'].append(skill)
        elif 'confl' in skill:
            skill_ba_jrp_groups['confluence (hard)'].append(skill)
        elif 'юнит' in skill or 'unit' in skill:
            skill_ba_jrp_groups['юнит-экономика (hard)'].append(skill)
        elif 'exc' in skill:
            skill_ba_jrp_groups['excel (hard)'].append(skill)
        elif 'баз' in skill or 'субд' in skill or 'base' in skill:
            skill_ba_jrp_groups['базы данных (hard)'].append(skill)
        elif 'англ' in skill or 'engl' in skill:
            skill_ba_jrp_groups['английский язык (hard)'].append(skill)
        elif 'презент' in skill:
            skill_ba_jrp_groups['презентация  (soft)'].append(skill)
        elif 'power' in skill and 'point' in skill:
            skill_ba_jrp_groups['powerpoint (hard)'].append(skill)
        elif ('активн' in skill) and ('клиент' not in skill):
            skill_ba_jrp_groups['активность  (soft)'].append(skill)
        elif ('команд' in skill) and ('управл' not in skill) and ('руковод' not in skill):
            skill_ba_jrp_groups['работа в команде  (soft)'].append(skill)
        elif 'крити' in skill:
            skill_ba_jrp_groups['критическое мышление  (soft)'].append(skill)
        elif 'реш' in skill:
            skill_ba_jrp_groups['принятие решений  (soft)'].append(skill)
        elif 'api' in skill:
            skill_ba_jrp_groups['api (hard)'].append(skill)
        elif 'риск' in skill or 'risk' in skill:
            skill_ba_jrp_groups['оценка рисков (hard)'].append(skill)
        elif 'креа' in skill:
            skill_ba_jrp_groups['креативность (soft)'].append(skill)
        elif 'показ' in skill or 'roi' in skill or 'cpl' in skill or 'cpa' in skill or 'cor' in skill or 'ltv' in skill:
            skill_ba_jrp_groups['бизнес-показатели (hard)'].append(skill)
        elif ('продукт' in skill or 'product' in skill) and '1с' not in skill:
            skill_ba_jrp_groups['product management (hard)'].append(skill)
        elif ('управл' in skill or 'лидер' in skill or 'руков' in skill or 'руков' in skill or 'организа' in skill or 'managem' in skill) and '1c' not in skill and '1с' not in skill and 'консульт' not in skill and 'поручен' not in skill and 'консульт' not in skill and 'внедрен' not in skill and '!с' not in skill and 'разраб' not in skill:
            skill_ba_jrp_groups['управление  (soft)'].append(skill)
        elif ('бизнес' in skill and 'анали' in skill) or ('busin' in skill and 'anal' in skill) or ('finan' in skill and 'anal' in skill):
            skill_ba_jrp_groups['бизнес-анализ (hard)'].append(skill)
        elif 'анали' in skill and 'мышл' in skill:
            skill_ba_jrp_groups['аналитическое мышление (soft)'].append(skill)
        elif 'bpmn' in skill:
            skill_ba_jrp_groups['bpmn (hard)'].append(skill)
        elif ('бизнес' in skill and 'процес' in skill) or ('busin' in skill and 'proc' in skill):
            skill_ba_jrp_groups['знание бизнес процессов (hard)'].append(skill)
        elif ('разраб' in skill and 'задан' in skill or 'тз' in skill or 'инструкц' in skill or 'реглам' in skill) and 'процес' not in skill and 'учет' not in skill:
            skill_ba_jrp_groups['разработка ТЗ, инструкций, регламентов (hard)'].append(skill)
        elif 'uml' in skill:
            skill_ba_jrp_groups['uml (hard)'].append(skill)
        elif 'jira' in skill:
            skill_ba_jrp_groups['jira (hard)'].append(skill)
        elif 'visio' in skill:
            skill_ba_jrp_groups['visio (hard)'].append(skill)
        elif 'автоматиз' in skill and 'процес' in skill:
            skill_ba_jrp_groups['автоматизация процессов (hard)'].append(skill)
        elif 'erp' in skill:
            skill_ba_jrp_groups['erp-системы (hard)'].append(skill)
        elif 'модел' in skill:
            skill_ba_jrp_groups['бизнес-моделирование (hard)'].append(skill)
        elif 'agile' in skill:
            skill_ba_jrp_groups['agile (hard)'].append(skill)
        elif 'сист' in skill and 'мышл' in skill:
            skill_ba_jrp_groups['системное мышление (soft)'].append(skill)
        elif '1c' in skill or '1с' in skill or '!c' in skill or '!с' in skill:
            skill_ba_jrp_groups['1с (hard)'].append(skill)
        elif 'scrum' in skill:
            skill_ba_jrp_groups['scrum (hard)'].append(skill)
        elif 'crm' in skill:
            skill_ba_jrp_groups['crm (hard)'].append(skill)
        elif 'idef' in skill:
            skill_ba_jrp_groups['idef0 (hard)'].append(skill)
        elif 'epc' in skill:
            skill_ba_jrp_groups['epc (hard)'].append(skill)
        elif 'бухг' in skill:
            skill_ba_jrp_groups['бухгалтерский учет (hard)'].append(skill)
        elif 'idef' in skill:
            skill_ba_jrp_groups['idef (hard)'].append(skill)
        elif 'прогноз' in skill:
            skill_ba_jrp_groups['прогнозирование (hard)'].append(skill)
        elif 'cjm' in skill:
            skill_ba_jrp_groups['cjm (hard)'].append(skill)
        elif 'user story' in skill:
            skill_ba_jrp_groups['user story (hard)'].append(skill)
        elif 'обуч' in skill or 'развит' in skill:
            skill_ba_jrp_groups['обучение и развитие (soft)'].append(skill)
        elif 'консульт' in skill or 'consult' in skill:
            skill_ba_jrp_groups['бизнес-консультирование (hard)'].append(skill)
        elif 'больш' in skill and 'инф' in skill:
            skill_ba_jrp_groups['работа с большим объемом информации (soft)'].append(skill)
        elif 'прин' in skill and 'реш' in skill:
            skill_ba_jrp_groups['принятие решений  (soft)'].append(skill)
        elif 'крит' in skill and 'мыш' in skill:
            skill_ba_jrp_groups['критическое мышление (soft)'].append(skill)
        elif 'sap' in skill:
            skill_ba_jrp_groups['sap (hard)'].append(skill)
        else:
            skill_ba_jrp_groups["other"].append(skill)


skills_ba_jrp_df = pd.DataFrame({"skill": list(skill_ba_jrp_groups.keys()), "count": [len(value) for value in skill_ba_jrp_groups.values()]})
skills_ba_jrp_df = skills_ba_jrp_df.sort_values(by='skill').reset_index(drop=True)
skills_ba_jrp_df.head()

,skill,count
0,1с (hard),307
1,agile (hard),20
2,api (hard),38
3,bpmn (hard),388
4,cjm (hard),15


##### Middle

In [ ]:
skill_ba_mid_count = {}

for index, row in ba.iterrows():
    if row['grade'] == 'middle':
        for skill in row['key_skills_process']:
            if skill in skill_ba_mid_count:
                skill_ba_mid_count[skill] += 1
            else:
                skill_ba_mid_count[skill] = 1

In [ ]:
#укрупним группы. За основу возьмём наиболее часто встречающиеся элементы
skill_ba_mid_groups = {'бизнес-анализ (hard)':[], 'аналитическое мышление (soft)': [], 'bpmn (hard)':[],
                'знание бизнес процессов (hard)':[], 'разработка ТЗ, инструкций, регламентов (hard)':[],
                'uml (hard)':[], 'sql (hard)':[],'документация (soft)': [],
                'коммуникация (soft)':[], 'data analysis (hard)':[],'бизнес-показатели (hard)':[], 'excel (hard)':[],
                'jira (hard)':[], 'visio (hard)':[], 'автоматизация процессов (hard)':[], 'confluence (hard)':[], 'powerpoint (hard)':[],
                'erp-системы (hard)':[], 'управление  (soft)':[], 'английский язык (hard)':[], 'работа в команде (soft)':[],
                'powerbi (hard)':[], 'бизнес-моделирование (hard)':[], 'agile (hard)':[], 'системное мышление (soft)':[], '1с (hard)':[], 'scrum (hard)':[],
                'активность  (soft)':[], 'crm (hard)':[], 'idef0 (hard)':[], 'python (hard)':[], 'базы данных (hard)':[], 'epc (hard)':[],
                'бухгалтерский учет (hard)':[], 'idef (hard)':[], 'презентация  (soft)':[], 'прогнозирование (hard)':[],
                'юнит-экономика (hard)':[], 'api (hard)':[],'креативность (soft)':[], 'cjm (hard)':[], 'user story (hard)':[],
                'обучение и развитие (soft)':[], 'tableau (hard)':[], 'бизнес-консультирование (hard)':[], 'оценка рисков (hard)':[],
                'работа с большим объемом информации (soft)':[], 'product management (hard)':[], 'принятие решений  (soft)':[],
                'статистика (hard)':[],'работа в команде  (soft)':[], 'критическое мышление  (soft)':[],
                'sap (hard)':[], 'other':[]}

In [ ]:
for skill, count in skill_ba_mid_count.items():
    for _ in range(count):
        if 'sql' in skill:
            skill_ba_mid_groups['sql (hard)'].append(skill)
        elif 'python' in skill or 'pandas' in skill or 'питон' in skill or 'scipy' in skill or 'plotly' in skill or 'seborn' in skill or 'matplotlib' in skill:
            skill_ba_mid_groups['python (hard)'].append(skill)
        elif ('analy' in skill or 'анали' in skill or 'гипот' in skill or 'olap' in skill) and 'business' not in skill and 'бизнес' not in skill and 'financial' not in skill and '1с аналитик' not in skill and 'мышл' not in skill:
            skill_ba_mid_groups['data analysis (hard)'].append(skill)
        elif ('документ' in skill or 'отчет' in skill or 'договор' in skill) and '1с: документооборот' not in skill and 'разраб' not in skill:
            skill_ba_mid_groups['документация (soft)'].append(skill)
        elif ('комму' in skill or 'речь' in skill or 'пис' in skill or 'грамот' in skill) and 'задан' not in skill and 'напис' not in skill:
            skill_ba_mid_groups['коммуникация (soft)'].append(skill)
        elif 'power' in skill and 'bi' in skill:
            skill_ba_mid_groups['powerbi (hard)'].append(skill)
        elif 'tableau' in skill:
            skill_ba_mid_groups['tableau (hard)'].append(skill)
        elif 'статис' in skill or 'statis' in skill:
            skill_ba_mid_groups['статистика (hard)'].append(skill)
        elif 'confl' in skill:
            skill_ba_mid_groups['confluence (hard)'].append(skill)
        elif 'юнит' in skill or 'unit' in skill:
            skill_ba_mid_groups['юнит-экономика (hard)'].append(skill)
        elif 'exc' in skill:
            skill_ba_mid_groups['excel (hard)'].append(skill)
        elif 'баз' in skill or 'субд' in skill or 'base' in skill:
            skill_ba_mid_groups['базы данных (hard)'].append(skill)
        elif 'англ' in skill or 'engl' in skill:
            skill_ba_mid_groups['английский язык (hard)'].append(skill)
        elif 'презент' in skill:
            skill_ba_mid_groups['презентация  (soft)'].append(skill)
        elif 'power' in skill and 'point' in skill:
            skill_ba_mid_groups['powerpoint (hard)'].append(skill)
        elif ('активн' in skill) and ('клиент' not in skill):
            skill_ba_mid_groups['активность  (soft)'].append(skill)
        elif ('команд' in skill) and ('управл' not in skill) and ('руковод' not in skill):
            skill_ba_mid_groups['работа в команде  (soft)'].append(skill)
        elif 'крити' in skill:
            skill_ba_mid_groups['критическое мышление  (soft)'].append(skill)
        elif 'реш' in skill:
            skill_ba_mid_groups['принятие решений  (soft)'].append(skill)
        elif 'api' in skill:
            skill_ba_mid_groups['api (hard)'].append(skill)
        elif 'риск' in skill or 'risk' in skill:
            skill_ba_mid_groups['оценка рисков (hard)'].append(skill)
        elif 'креа' in skill:
            skill_ba_mid_groups['креативность (soft)'].append(skill)
        elif 'показ' in skill or 'roi' in skill or 'cpl' in skill or 'cpa' in skill or 'cor' in skill or 'ltv' in skill:
            skill_ba_mid_groups['бизнес-показатели (hard)'].append(skill)
        elif ('продукт' in skill or 'product' in skill) and '1с' not in skill:
            skill_ba_mid_groups['product management (hard)'].append(skill)
        elif ('управл' in skill or 'лидер' in skill or 'руков' in skill or 'руков' in skill or 'организа' in skill or 'managem' in skill) and '1c' not in skill and '1с' not in skill and 'консульт' not in skill and 'поручен' not in skill and 'консульт' not in skill and 'внедрен' not in skill and '!с' not in skill and 'разраб' not in skill:
            skill_ba_mid_groups['управление  (soft)'].append(skill)
        elif ('бизнес' in skill and 'анали' in skill) or ('busin' in skill and 'anal' in skill) or ('finan' in skill and 'anal' in skill):
            skill_ba_mid_groups['бизнес-анализ (hard)'].append(skill)
        elif 'анали' in skill and 'мышл' in skill:
            skill_ba_mid_groups['аналитическое мышление (soft)'].append(skill)
        elif 'bpmn' in skill:
            skill_ba_mid_groups['bpmn (hard)'].append(skill)
        elif ('бизнес' in skill and 'процес' in skill) or ('busin' in skill and 'proc' in skill):
            skill_ba_mid_groups['знание бизнес процессов (hard)'].append(skill)
        elif ('разраб' in skill and 'задан' in skill or 'тз' in skill or 'инструкц' in skill or 'реглам' in skill) and 'процес' not in skill and 'учет' not in skill:
            skill_ba_mid_groups['разработка ТЗ, инструкций, регламентов (hard)'].append(skill)
        elif 'uml' in skill:
            skill_ba_mid_groups['uml (hard)'].append(skill)
        elif 'jira' in skill:
            skill_ba_mid_groups['jira (hard)'].append(skill)
        elif 'visio' in skill:
            skill_ba_mid_groups['visio (hard)'].append(skill)
        elif 'автоматиз' in skill and 'процес' in skill:
            skill_ba_mid_groups['автоматизация процессов (hard)'].append(skill)
        elif 'erp' in skill:
            skill_ba_mid_groups['erp-системы (hard)'].append(skill)
        elif 'модел' in skill:
            skill_ba_mid_groups['бизнес-моделирование (hard)'].append(skill)
        elif 'agile' in skill:
            skill_ba_mid_groups['agile (hard)'].append(skill)
        elif 'сист' in skill and 'мышл' in skill:
            skill_ba_mid_groups['системное мышление (soft)'].append(skill)
        elif '1c' in skill or '1с' in skill or '!c' in skill or '!с' in skill:
            skill_ba_mid_groups['1с (hard)'].append(skill)
        elif 'scrum' in skill:
            skill_ba_mid_groups['scrum (hard)'].append(skill)
        elif 'crm' in skill:
            skill_ba_mid_groups['crm (hard)'].append(skill)
        elif 'idef' in skill:
            skill_ba_mid_groups['idef0 (hard)'].append(skill)
        elif 'epc' in skill:
            skill_ba_mid_groups['epc (hard)'].append(skill)
        elif 'бухг' in skill:
            skill_ba_mid_groups['бухгалтерский учет (hard)'].append(skill)
        elif 'idef' in skill:
            skill_ba_mid_groups['idef (hard)'].append(skill)
        elif 'прогноз' in skill:
            skill_ba_mid_groups['прогнозирование (hard)'].append(skill)
        elif 'cjm' in skill:
            skill_ba_mid_groups['cjm (hard)'].append(skill)
        elif 'user story' in skill:
            skill_ba_mid_groups['user story (hard)'].append(skill)
        elif 'обуч' in skill or 'развит' in skill:
            skill_ba_mid_groups['обучение и развитие (soft)'].append(skill)
        elif 'консульт' in skill or 'consult' in skill:
            skill_ba_mid_groups['бизнес-консультирование (hard)'].append(skill)
        elif 'больш' in skill and 'инф' in skill:
            skill_ba_mid_groups['работа с большим объемом информации (soft)'].append(skill)
        elif 'прин' in skill and 'реш' in skill:
            skill_ba_mid_groups['принятие решений  (soft)'].append(skill)
        elif 'крит' in skill and 'мыш' in skill:
            skill_ba_mid_groups['критическое мышление (soft)'].append(skill)
        elif 'sap' in skill:
            skill_ba_mid_groups['sap (hard)'].append(skill)
        else:
            skill_ba_mid_groups["other"].append(skill)


skills_ba_mid_df = pd.DataFrame({"skill": list(skill_ba_mid_groups.keys()), "count": [len(value) for value in skill_ba_mid_groups.values()]})
skills_ba_mid_df = skills_ba_mid_df.sort_values(by='skill').reset_index(drop=True)
skills_ba_mid_df.head()

,skill,count
0,1с (hard),274
1,agile (hard),19
2,api (hard),20
3,bpmn (hard),342
4,cjm (hard),19


##### Senior

In [ ]:
skill_ba_sen_count = {}

for index, row in ba.iterrows():
    if row['grade'] == 'senior':
        for skill in row['key_skills_process']:
            if skill in skill_ba_sen_count:
                skill_ba_sen_count[skill] += 1
            else:
                skill_ba_sen_count[skill] = 1

In [ ]:
#укрупним группы. За основу возьмём наиболее часто встречающиеся элементы
skill_ba_sen_groups = {'бизнес-анализ (hard)':[], 'аналитическое мышление (soft)': [], 'bpmn (hard)':[],
                'знание бизнес процессов (hard)':[], 'разработка ТЗ, инструкций, регламентов (hard)':[],
                'uml (hard)':[], 'sql (hard)':[],'документация (soft)': [],
                'коммуникация (soft)':[], 'data analysis (hard)':[],'бизнес-показатели (hard)':[], 'excel (hard)':[],
                'jira (hard)':[], 'visio (hard)':[], 'автоматизация процессов (hard)':[], 'confluence (hard)':[], 'powerpoint (hard)':[],
                'erp-системы (hard)':[], 'управление  (soft)':[], 'английский язык (hard)':[], 'работа в команде (soft)':[],
                'powerbi (hard)':[], 'бизнес-моделирование (hard)':[], 'agile (hard)':[], 'системное мышление (soft)':[], '1с (hard)':[], 'scrum (hard)':[],
                'активность  (soft)':[], 'crm (hard)':[], 'idef0 (hard)':[], 'python (hard)':[], 'базы данных (hard)':[], 'epc (hard)':[],
                'бухгалтерский учет (hard)':[], 'idef (hard)':[], 'презентация  (soft)':[], 'прогнозирование (hard)':[],
                'юнит-экономика (hard)':[], 'api (hard)':[],'креативность (soft)':[], 'cjm (hard)':[], 'user story (hard)':[],
                'обучение и развитие (soft)':[], 'tableau (hard)':[], 'бизнес-консультирование (hard)':[], 'оценка рисков (hard)':[],
                'работа с большим объемом информации (soft)':[], 'product management (hard)':[], 'принятие решений  (soft)':[],
                'статистика (hard)':[],'работа в команде  (soft)':[], 'критическое мышление  (soft)':[],
                'sap (hard)':[], 'other':[]}

In [ ]:
for skill, count in skill_ba_sen_count.items():
    for _ in range(count):
        if 'sql' in skill:
            skill_ba_sen_groups['sql (hard)'].append(skill)
        elif 'python' in skill or 'pandas' in skill or 'питон' in skill or 'scipy' in skill or 'plotly' in skill or 'seborn' in skill or 'matplotlib' in skill:
            skill_ba_sen_groups['python (hard)'].append(skill)
        elif ('analy' in skill or 'анали' in skill or 'гипот' in skill or 'olap' in skill) and 'business' not in skill and 'бизнес' not in skill and 'financial' not in skill and '1с аналитик' not in skill and 'мышл' not in skill:
            skill_ba_sen_groups['data analysis (hard)'].append(skill)
        elif ('документ' in skill or 'отчет' in skill or 'договор' in skill) and '1с: документооборот' not in skill and 'разраб' not in skill:
            skill_ba_sen_groups['документация (soft)'].append(skill)
        elif ('комму' in skill or 'речь' in skill or 'пис' in skill or 'грамот' in skill) and 'задан' not in skill and 'напис' not in skill:
            skill_ba_sen_groups['коммуникация (soft)'].append(skill)
        elif 'power' in skill and 'bi' in skill:
            skill_ba_sen_groups['powerbi (hard)'].append(skill)
        elif 'tableau' in skill:
            skill_ba_sen_groups['tableau (hard)'].append(skill)
        elif 'статис' in skill or 'statis' in skill:
            skill_ba_sen_groups['статистика (hard)'].append(skill)
        elif 'confl' in skill:
            skill_ba_sen_groups['confluence (hard)'].append(skill)
        elif 'юнит' in skill or 'unit' in skill:
            skill_ba_sen_groups['юнит-экономика (hard)'].append(skill)
        elif 'exc' in skill:
            skill_ba_sen_groups['excel (hard)'].append(skill)
        elif 'баз' in skill or 'субд' in skill or 'base' in skill:
            skill_ba_sen_groups['базы данных (hard)'].append(skill)
        elif 'англ' in skill or 'engl' in skill:
            skill_ba_sen_groups['английский язык (hard)'].append(skill)
        elif 'презент' in skill:
            skill_ba_sen_groups['презентация  (soft)'].append(skill)
        elif 'power' in skill and 'point' in skill:
            skill_ba_sen_groups['powerpoint (hard)'].append(skill)
        elif ('активн' in skill) and ('клиент' not in skill):
            skill_ba_sen_groups['активность  (soft)'].append(skill)
        elif ('команд' in skill) and ('управл' not in skill) and ('руковод' not in skill):
            skill_ba_sen_groups['работа в команде  (soft)'].append(skill)
        elif 'крити' in skill:
            skill_ba_sen_groups['критическое мышление  (soft)'].append(skill)
        elif 'реш' in skill:
            skill_ba_sen_groups['принятие решений  (soft)'].append(skill)
        elif 'api' in skill:
            skill_ba_sen_groups['api (hard)'].append(skill)
        elif 'риск' in skill or 'risk' in skill:
            skill_ba_sen_groups['оценка рисков (hard)'].append(skill)
        elif 'креа' in skill:
            skill_ba_sen_groups['креативность (soft)'].append(skill)
        elif 'показ' in skill or 'roi' in skill or 'cpl' in skill or 'cpa' in skill or 'cor' in skill or 'ltv' in skill:
            skill_ba_sen_groups['бизнес-показатели (hard)'].append(skill)
        elif ('продукт' in skill or 'product' in skill) and '1с' not in skill:
            skill_ba_sen_groups['product management (hard)'].append(skill)
        elif ('управл' in skill or 'лидер' in skill or 'руков' in skill or 'руков' in skill or 'организа' in skill or 'managem' in skill) and '1c' not in skill and '1с' not in skill and 'консульт' not in skill and 'поручен' not in skill and 'консульт' not in skill and 'внедрен' not in skill and '!с' not in skill and 'разраб' not in skill:
            skill_ba_sen_groups['управление  (soft)'].append(skill)
        elif ('бизнес' in skill and 'анали' in skill) or ('busin' in skill and 'anal' in skill) or ('finan' in skill and 'anal' in skill):
            skill_ba_sen_groups['бизнес-анализ (hard)'].append(skill)
        elif 'анали' in skill and 'мышл' in skill:
            skill_ba_sen_groups['аналитическое мышление (soft)'].append(skill)
        elif 'bpmn' in skill:
            skill_ba_sen_groups['bpmn (hard)'].append(skill)
        elif ('бизнес' in skill and 'процес' in skill) or ('busin' in skill and 'proc' in skill):
            skill_ba_sen_groups['знание бизнес процессов (hard)'].append(skill)
        elif ('разраб' in skill and 'задан' in skill or 'тз' in skill or 'инструкц' in skill or 'реглам' in skill) and 'процес' not in skill and 'учет' not in skill:
            skill_ba_sen_groups['разработка ТЗ, инструкций, регламентов (hard)'].append(skill)
        elif 'uml' in skill:
            skill_ba_sen_groups['uml (hard)'].append(skill)
        elif 'jira' in skill:
            skill_ba_sen_groups['jira (hard)'].append(skill)
        elif 'visio' in skill:
            skill_ba_sen_groups['visio (hard)'].append(skill)
        elif 'автоматиз' in skill and 'процес' in skill:
            skill_ba_sen_groups['автоматизация процессов (hard)'].append(skill)
        elif 'erp' in skill:
            skill_ba_sen_groups['erp-системы (hard)'].append(skill)
        elif 'модел' in skill:
            skill_ba_sen_groups['бизнес-моделирование (hard)'].append(skill)
        elif 'agile' in skill:
            skill_ba_sen_groups['agile (hard)'].append(skill)
        elif 'сист' in skill and 'мышл' in skill:
            skill_ba_sen_groups['системное мышление (soft)'].append(skill)
        elif '1c' in skill or '1с' in skill or '!c' in skill or '!с' in skill:
            skill_ba_sen_groups['1с (hard)'].append(skill)
        elif 'scrum' in skill:
            skill_ba_sen_groups['scrum (hard)'].append(skill)
        elif 'crm' in skill:
            skill_ba_sen_groups['crm (hard)'].append(skill)
        elif 'idef' in skill:
            skill_ba_sen_groups['idef0 (hard)'].append(skill)
        elif 'epc' in skill:
            skill_ba_sen_groups['epc (hard)'].append(skill)
        elif 'бухг' in skill:
            skill_ba_sen_groups['бухгалтерский учет (hard)'].append(skill)
        elif 'idef' in skill:
            skill_ba_sen_groups['idef (hard)'].append(skill)
        elif 'прогноз' in skill:
            skill_ba_sen_groups['прогнозирование (hard)'].append(skill)
        elif 'cjm' in skill:
            skill_ba_sen_groups['cjm (hard)'].append(skill)
        elif 'user story' in skill:
            skill_ba_sen_groups['user story (hard)'].append(skill)
        elif 'обуч' in skill or 'развит' in skill:
            skill_ba_sen_groups['обучение и развитие (soft)'].append(skill)
        elif 'консульт' in skill or 'consult' in skill:
            skill_ba_sen_groups['бизнес-консультирование (hard)'].append(skill)
        elif 'больш' in skill and 'инф' in skill:
            skill_ba_sen_groups['работа с большим объемом информации (soft)'].append(skill)
        elif 'прин' in skill and 'реш' in skill:
            skill_ba_sen_groups['принятие решений  (soft)'].append(skill)
        elif 'крит' in skill and 'мыш' in skill:
            skill_ba_sen_groups['критическое мышление (soft)'].append(skill)
        elif 'sap' in skill:
            skill_ba_sen_groups['sap (hard)'].append(skill)
        else:
            skill_ba_sen_groups["other"].append(skill)


skills_ba_sen_df = pd.DataFrame({"skill": list(skill_ba_sen_groups.keys()), "count": [len(value) for value in skill_ba_sen_groups.values()]})
skills_ba_sen_df = skills_ba_sen_df.sort_values(by='skill').reset_index(drop=True)
skills_ba_sen_df.head()

,skill,count
0,1с (hard),10
1,agile (hard),1
2,api (hard),8
3,bpmn (hard),21
4,cjm (hard),0


#### Объединённые данные по скиллам BA

In [ ]:
#объединим получившиеся данные и разделим на два датафрейма: хард и софт скиллы
skills_final_ba = pd.merge(skills_ba_df, skills_ba_jr_df, on='skill')
skills_final_ba = pd.merge(skills_final_ba, skills_ba_jrp_df, on='skill')
skills_final_ba = pd.merge(skills_final_ba, skills_ba_mid_df, on='skill')
skills_final_ba = pd.merge(skills_final_ba, skills_ba_sen_df, on='skill').sort_values(by='count', ascending=False).reset_index(drop=True)

skills_final_ba.columns = ['skill', 'total', 'jr', 'jr+', 'mid', 'sen']

<ipython-input-83-505d95f59a2f>:4: FutureWarning: Passing 'suffixes' which cause duplicate columns {'count_x'} in the result is deprecated and will raise a MergeError in a future version.
  skills_final_ba = pd.merge(skills_final_ba, skills_ba_mid_df, on='skill')


In [ ]:
hard_skills_final_ba = skills_final_ba[skills_final_ba['skill'].str.contains('hard')].reset_index(drop=True)
hard_skills_final_ba

,skill,total,jr,jr+,mid,sen
0,бизнес-анализ (hard),1682,151,859,632,40
1,data analysis (hard),1682,163,886,602,31
2,sql (hard),706,59,351,270,26
3,знание бизнес процессов (hard),1161,79,614,446,22
4,bpmn (hard),791,40,388,342,21
5,confluence (hard),783,45,377,346,15
6,uml (hard),393,42,182,157,12
7,1с (hard),765,174,307,274,10
8,"разработка ТЗ, инструкций, регламентов (hard)",674,38,408,219,9
9,api (hard),67,1,38,20,8


In [ ]:
hard_skills_final_ba.to_csv('hard_skills_final_ba.csv', index = False)

In [ ]:
soft_skills_final_ba = skills_final_ba[skills_final_ba['skill'].str.contains('soft')].reset_index(drop=True)
soft_skills_final_ba

,skill,total,jr,jr+,mid,sen
0,документация (soft),2247,151,1177,869,50
1,управление (soft),621,66,304,228,23
2,коммуникация (soft),1174,127,598,426,23
3,презентация (soft),232,15,131,76,10
4,аналитическое мышление (soft),1027,158,525,338,6
5,активность (soft),93,1,35,53,4
6,обучение и развитие (soft),58,7,32,16,3
7,системное мышление (soft),109,12,49,46,2
8,работа с большим объемом информации (soft),306,38,161,105,2
9,работа в команде (soft),135,13,84,36,2


In [ ]:
soft_skills_final_ba.to_csv('soft_skills_final_ba.csv', index = False)

### Требуемый опыт

In [ ]:
#сгруппируем по грейдам имеющиеся данные
exp_req_ba = ba[['grade','experience_year']]
exp_req_ba['count']=''

exp_req_ba = exp_req_ba.pivot_table(index= 'grade', columns='experience_year', values='experience_year', aggfunc='count').reset_index()
exp_req_ba = exp_req_ba.rename_axis(None, axis=1)
exp_req_ba.transpose()

<ipython-input-88-c84992df2837>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exp_req_ba['count']=''


,0,1,2,3
grade,junior,junior+,middle,senior
1-3,4.0,2113.0,4.0,1.0
3-6,NaN,NaN,1595.0,1.0
6+,NaN,NaN,1.0,88.0
no experince,364.0,NaN,NaN,NaN


In [ ]:
exp_req_ba.to_csv('exp_req_ba.csv', index = False)

### Определение типичного места работы BA

#### Топ работодателей

In [ ]:
employer_ba_total = ba.groupby('employer')['id'].count().sort_values(ascending=False)
employer_ba_jr = ba[ba['grade'] == 'junior'].groupby('employer')['id'].count()
employer_ba_jrp = ba[ba['grade'] == 'junior+'].groupby('employer')['id'].count()
employer_ba_mid = ba[ba['grade'] == 'middle'].groupby('employer')['id'].count()
employer_ba_sen = ba[ba['grade'] == 'senior'].groupby('employer')['id'].count()

employer_ba = pd.concat([employer_ba_total, employer_ba_jr, employer_ba_jrp, employer_ba_mid, employer_ba_sen], axis=1).reset_index()
employer_ba.columns = ['employer', 'total', 'jr', 'jr+', 'mid', 'sen']
employer_ba.head()

,employer,total,jr,jr+,mid,sen
0,Банк ВТБ (ПАО),88,NaN,46.0,42.0,NaN
1,Aston,66,59.0,3.0,4.0,NaN
2,СБЕР,58,3.0,36.0,18.0,1.0
3,"МАГНИТ, Розничная сеть",56,6.0,37.0,13.0,NaN
4,Комус,38,1.0,19.0,17.0,1.0


In [ ]:
employer_ba.to_csv('employer_ba.csv', index = False)

#### Зарплата

In [ ]:
#изучим данные по ЗП у джунов без опыта
salary_jr_ba = ba[ba['grade'] == 'junior'].describe()
salary_jr_ba = salary_jr_ba.drop(['id'], axis=1)
salary_jr_ba = salary_jr_ba.drop(['std', '25%', '75%'], axis=0).reset_index()
salary_jr_ba_raw = ba[ba['grade'] == 'junior']
salary_jr_ba_raw = salary_jr_ba_raw[['id','salary_from', 'salary_to']].reset_index(drop=True)
salary_jr_ba

,index,salary_from,salary_to
0,count,158.000000,130.000000
1,mean,55098.101266,82033.846154
2,min,20000.000000,30000.000000
3,50%,50000.000000,80000.000000
4,max,150000.000000,200000.000000


In [ ]:
salary_jr_ba_raw.to_csv('salary_jr_ba_raw.csv', index = False)

In [ ]:
salary_jr_ba.to_csv('salary_jr_ba.csv', index = False)

In [ ]:
#изучим данные по ЗП у джунов с опытом 1-3 года
salary_jrp_ba = ba.drop(ba[ba['salary_from'] < 20000].index)
salary_jrp_ba_raw = salary_jrp_ba[salary_jrp_ba['grade'] == 'junior+']
salary_jrp_ba_raw = salary_jrp_ba_raw[['id','salary_from', 'salary_to']].reset_index(drop=True)
salary_jrp_ba = salary_jrp_ba[salary_jrp_ba['grade'] == 'junior+'].describe()
salary_jrp_ba = salary_jrp_ba.drop(['id'], axis=1)
salary_jrp_ba = salary_jrp_ba.drop(['std', '25%', '75%'], axis=0).reset_index()

salary_jrp_ba

,index,salary_from,salary_to
0,count,506.000000,307.000000
1,mean,100908.841897,140637.583062
2,min,30000.000000,45000.000000
3,50%,85000.000000,120000.000000
4,max,350000.000000,400000.000000


In [ ]:
salary_jrp_ba_raw.to_csv('salary_jrp_ba_raw.csv', index = False)
salary_jrp_ba.to_csv('salary_jrp_ba.csv', index = False)

In [ ]:
#изучим данные по ЗП у миддлов
salary_mid_ba = ba[ba['grade'] == 'middle'].describe()
salary_mid_ba = salary_mid_ba.drop(['id'], axis=1)
salary_mid_ba = salary_mid_ba.drop(['std', '25%', '75%'], axis=0).reset_index()
salary_mid_ba_raw = ba[ba['grade'] == 'middle']
salary_mid_ba_raw = salary_mid_ba_raw[['id','salary_from', 'salary_to']].reset_index(drop=True)
salary_mid_ba

,index,salary_from,salary_to
0,count,246.000000,181.000000
1,mean,156884.349593,214066.850829
2,min,45000.000000,60000.000000
3,50%,150000.000000,200000.000000
4,max,350000.000000,400000.000000


In [ ]:
salary_mid_ba_raw.to_csv('salary_mid_ba_raw.csv', index = False)
salary_mid_ba.to_csv('salary_mid_ba.csv', index = False)

In [ ]:
#изучим данные по ЗП у синьйоров
salary_sen_ba = ba[ba['grade'] == 'senior'].describe()
salary_sen_ba = salary_sen_ba.drop(['id'], axis=1)
salary_sen_ba = salary_sen_ba.drop(['std', '25%', '75%'], axis=0).reset_index()
salary_sen_ba_raw = ba[ba['grade'] == 'senior']
salary_sen_ba_raw = salary_sen_ba_raw[['id','salary_from', 'salary_to']].reset_index(drop=True)
salary_sen_ba

,index,salary_from,salary_to
0,count,16.00,14.000000
1,mean,219406.25,287678.571429
2,min,100000.00,200000.000000
3,50%,225000.00,265000.000000
4,max,330000.00,467500.000000


In [ ]:
salary_sen_ba_raw.to_csv('salary_sen_ba_raw.csv', index = False)
salary_sen_ba.to_csv('salary_sen_ba.csv', index = False)

#### Тип занятости

In [ ]:
employment_ba_jr = ba[ba['grade'] == 'junior'].groupby('employment')['id'].count()
employment_ba_jrp = ba[ba['grade'] == 'junior+'].groupby('employment')['id'].count()
employment_ba_mid = ba[ba['grade'] == 'middle'].groupby('employment')['id'].count()
employment_ba_sen = ba[ba['grade'] == 'senior'].groupby('employment')['id'].count()
employment_ba = pd.concat([employment_ba_jr, employment_ba_jrp, employment_ba_mid, employment_ba_sen], axis=1).reset_index()
employment_ba = employment_ba.fillna(0)
employment_ba.columns = ['Тип занятости', 'jr', 'jr+', 'mid', 'sen']
employment_ba

,Тип занятости,jr,jr+,mid,sen
0,Полная занятость,252,2093,1589,90.0
1,Проектная работа,2,9,8,0.0
2,Стажировка,113,6,1,0.0
3,Частичная занятость,1,5,2,0.0


In [ ]:
employment_ba.to_csv('employment_ba.csv', index = False)

#### График работы

In [ ]:
schedule_ba_jr = ba[ba['grade'] == 'junior'].groupby('schedule')['id'].count()
schedule_ba_jrp = ba[ba['grade'] == 'junior+'].groupby('schedule')['id'].count()
schedule_ba_mid = ba[ba['grade'] == 'middle'].groupby('schedule')['id'].count()
schedule_ba_sen = ba[ba['grade'] == 'senior'].groupby('schedule')['id'].count()
schedule_ba = pd.concat([schedule_ba_jr, schedule_ba_jrp, schedule_ba_mid, schedule_ba_sen], axis=1).reset_index()
schedule_ba = schedule_ba.fillna(0)
schedule_ba.columns = ['Расписание', 'jr', 'jr+', 'mid', 'sen']
schedule_ba

,Расписание,jr,jr+,mid,sen
0,Гибкий график,19.0,52,41.0,3.0
1,Полный день,315.0,1730,1229.0,61.0
2,Удаленная работа,34.0,327,330.0,26.0
3,Сменный график,0.0,4,0.0,0.0


In [ ]:
schedule_ba.to_csv('schedule_ba.csv', index = False)

### Помесячная динамика количества вакансий для BA

In [ ]:
vac_ba_jr = ba[ba['grade'] == 'junior'].groupby('month')['id'].count()
vac_ba_jrp = ba[ba['grade'] == 'junior+'].groupby('month')['id'].count()
vac_ba_mid = ba[ba['grade'] == 'middle'].groupby('month')['id'].count()
vac_ba_sen = ba[ba['grade'] == 'senior'].groupby('month')['id'].count()
vac_ba = pd.concat([vac_ba_jr, vac_ba_jrp, vac_ba_mid, vac_ba_sen], axis=1).reset_index()
vac_ba.columns = ['Месяц', 'jr', 'jr+', 'mid', 'sen']
vac_ba

,Месяц,jr,jr+,mid,sen
0,01/2024,146,899,725,42
1,10/2023,24,159,103,6
2,11/2023,125,601,441,20
3,12/2023,73,454,331,22


In [ ]:
vac_ba.to_csv('vac_ba.csv', index = False)

Выводы и рекомендации представлены в прилагаемой презентации.